In [1]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic



In [ ]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, SEED,RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, EVENT, UNIQUE_ID, DOB, COUNTRY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, REGION
FROM `saa-analytics.results.saa_full` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL
"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




In [700]:
competitors.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)

In [701]:
competitors

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,None,Mid,Female,SEA Youth,2024,International
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43941,"Guk, Jean",,31:29.71,J Division,41.0,4,Women 5000 Meter Run Women Senior B,None,1982-09-12,None,,Long,Female,71st PSA Track & Field Championship 2023,2023,Local
43942,"Ho, Xiuping",,17.93,Training Command,42.0,4,Women 100 Meter Dash Women Senior B,None,1981-12-10,None,1.1,Sprint,Female,71st PSA Track & Field Championship 2023,2023,Local
43943,"Toh, Pei Lian",,17.87,Training Command,45.0,3,Women 100 Meter Dash Women Senior B,None,1978-10-03,None,1.1,Sprint,Female,71st PSA Track & Field Championship 2023,2023,Local
43944,"Lin, Cheng Fong",,21.94,Police Coast Guard,44.0,7,Women 100 Meter Dash Women Senior B,None,1979-07-29,None,1.1,Sprint,Female,71st PSA Track & Field Championship 2023,2023,Local


In [1282]:
athletes_2024 = competitors[competitors['DATE']=='2024']

In [1283]:
athletes_2024

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,None,Mid,Female,SEA Youth,2024,International
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43688,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43689,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43690,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43691,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local


In [1284]:
athletes_2024[athletes_2024['COMPETITION']=='13th ASEAN Schools Games']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION


In [1285]:
athletes_2024.to_csv('athletes_2024.csv', encoding='utf-8')

In [1286]:
# Choose 2024 only

athletes = athletes_2024

In [1287]:
athletes

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,None,Mid,Female,SEA Youth,2024,International
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43688,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43689,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43690,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local
43691,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local


In [1288]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION
474,Jun Jie Calvin Quek,None,51.8,Singapore,None,1,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Taiwan Athletics Open,2024,International
494,Jun Jie Calvin Quek,None,51.79,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International
501,Jun Jie Calvin Quek,None,53.19,Singapore,None,4,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,UAE Athletics Grand Prix,2024,International
616,Jun Jie Calvin Quek,None,52.05,Singapore,None,8,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,11th Kinami Michitaka Memorial Athletics Meet,2024,International
626,Jun Jie Calvin Quek,None,52.18,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International
1125,Jun Jie Calvin Quek,None,56.95,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International
1209,Jun Jie Calvin Quek,None,51.33,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International


In [1289]:
# Run events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m')

# Hurdles events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110m Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Hurdles.+', value='200m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400m Hurdles.+', value='400m hurdles')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Race Walk.+', value='1500m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Race Walk.+', value='10000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

#mask = athletes['EVENT'].str.contains(r'Discus', na=True)
#athletes.loc[mask, 'EVENT'] = 'Discus throw'

#mask = athletes['EVENT'].str.contains(r'Shot', na=True)
#athletes.loc[mask, 'EVENT'] = 'Shot put'

#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'EVENT'] = 'Javelin throw'



#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+2000 Meter Steeplechase.+', value='2000m steeplechase')


#mask = athletes['EVENT'].str.contains(r'High', na=True)
#athletes.loc[mask, 'EVENT'] = 'High jump'

#mask = athletes['EVENT'].str.contains(r'110m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '110m hurdles'

#mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '400m hurdles'

#mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '200m hurdles'

#mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '100m hurdles'

#mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
#athletes.loc[mask, 'EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'


#mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'


# correct javelin category
#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1290]:
# Wind mapping

#mask = athletes['EVENT'].str.contains(r'\d{1}.\d{1}', na=True, regex=True)
#athletes.loc[mask, 'WIND_MAP'] = '400m'


In [1291]:
# Remove special characters before mapping

athletes['EVENT'] = athletes['EVENT'].str.replace('\xa0', '', regex=True)
athletes['EVENT'] = athletes['EVENT'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
athletes['EVENT'] = athletes['EVENT'].str.replace('\r', '', regex=True)
athletes['EVENT'] = athletes['EVENT'].str.replace('\n', '', regex=True)
athletes['EVENT'] = athletes['EVENT'].str.strip()


# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'

mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^100m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'^100\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'

mask = athletes['EVENT'].str.contains(r'^110\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'^110m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'


#mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
#mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'



mask = athletes['EVENT'].str.contains(r'^200m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200m\sHurdles\s\(0\.762m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '



mask = athletes['EVENT'].str.contains(r'400m\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'

mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'^Javelin\sThrow$', na=True, regex=True)  # exact match for string
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sU18', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'



mask = athletes['EVENT'].str.contains(r'^Shot\sPut$', na=True, regex=True) # there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'

mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4\.00kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'

mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
#mask = athletes['EVENT'].str.contains(r'Shot Put Masters', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'^Hammer\sThrow$', na=True)  # there are some characters after Throw
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'

mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'
mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'^Discus\sThrow    $', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'^Discus\sThrow$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'

mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'

mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=True))  & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



mask = athletes['EVENT'].str.contains(r'Women\sDiscus\sThrow\s\(1kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOPEN', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'

# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half marathon'

# Walk

mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m race walk'
mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m race walk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m race walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/3562864066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['EVENT'] = athletes['EVENT'].str.replace('\xa0', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/3562864066.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['EVENT'] = athletes['EVENT'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/3562864066.py:5: SettingWit

In [1292]:
athletes

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION,MAPPED_EVENT
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,None,Mid,Female,SEA Youth,2024,International,1500m
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International,Pole vault
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International,Pole vault
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International,2000m steeplechase
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International,2000m steeplechase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43688,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43689,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43690,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43691,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,


In [1293]:
athletes[athletes['MAPPED_EVENT']=='200m']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION,MAPPED_EVENT
9,YEE CHERN CLARA LIM,None,26.76,Singapore,None,7,200m,None,2006,SGP,-1.3,Sprint,Female,SEA Youth,2024,International,200m
13,TEO ALEXIS PEI QI,None,27.15,Singapore,None,5,200m,None,2007,SGP,-0.1,Sprint,Female,SEA Youth,2024,International,200m
389,Shanti Veronica Pereira,None,23.56,Singapore,None,2,200m,None,20-Sep-96,nan,-2.6,Sprint,Female,Motonet GP,2024,International,200m
404,Subaraghav Hari,None,22.52,Singapore,None,1,200m,None,7-Jan-06,nan,Wind: -0.6,Sprint,Male,Nagase Cup,2024,International,200m
405,Xander Ho Ann Heng,None,22.66,Singapore,None,4,200m,None,19-May-00,nan,Wind: 0.3,Sprint,Male,Nagase Cup,2024,International,200m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43348,"Naito, Kano",,26.59,Waseda,20.0,4,Women 200 Meter Dash Invitational,None,2004-06-07,None,0.6,Sprint,Female,SA Allcomers Meet 3,2024,Local,200m
43349,"Sagi, Mayako",,24.48,Waseda,21.0,1,Women 200 Meter Dash Invitational,None,2003-01-03,None,0.6,Sprint,Female,SA Allcomers Meet 3,2024,Local,200m
43359,"Inage, Aoi",,21.37,Waseda,23.0,4,Men 200 Meter Dash Invitational,None,2001-04-06,None,-0.3,Sprint,Male,SA Allcomers Meet 3,2024,Local,200m
43360,"Kimura, Ryo",,21.31,Meiji University,23.0,3,Men 200 Meter Dash Invitational,None,2001-07-07,None,-0.3,Sprint,Male,SA Allcomers Meet 3,2024,Local,200m


In [1294]:
athletes.to_csv('athletes_post_map.csv', sep=',', encoding='utf-8-sig', index=False)


In [1295]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [1298]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3'
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1299]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final
1,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final
2,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final
3,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final
4,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final
5,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final
6,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final
7,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final
8,Goh Chui Ling,4:26.33,3,1500m,Mid,Female,SEA Games,Final
9,Robi Syianturi,14:43.45,3,5000m,Long,Male,SEA Games,Final


In [1300]:
benchmarks=SEAG

In [1301]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [1302]:
benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [1303]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,11.75,100m,Female
1,10.443,100m,Male
2,21.02,200m,Male
3,23.6,200m,Female
4,46.63,400m,Male
5,53.84,400m,Female
6,2:09.15,800m,Female
7,1:53.86,800m,Male
8,4:26.33,1500m,Female
9,14:43.45,5000m,Male


In [1304]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
    print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
            print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
          #  elif 'w' in metric:
                
               
           #     metric1=metric.replace('w', '')
                
           #     output=float(str(metric1))
                
                else:
    
                    output=float(str(metric))
        
        #elif 'w' in metric:
            
                
         #       metric1=metric.replace('w', '')
                             
         #       output=float(str(metric1))
                
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
      #      elif 'marathon' in string and count==2:
                
      #          print('HERE', metric)
                
      #          metric = metric.replace(":", ".", 2)
                
      #          print('METRIC', metric)
                
       #         h,m,s = metric.split(':')            

       #         output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())

               
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [1305]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

metric 11.75
metric 10.443
metric 21.02
metric 23.6
metric 46.63
metric 53.84
metric 2:09.15
metric 1:53.86
metric 4:26.33
metric 14:43.45
metric 17:13.63
metric 35:31.03
metric 31:25.55
metric 2:35:49
metric 2:50:27
metric 17.3
metric 14.44
metric 03:59.40
metric 6891
metric 1.73
metric 2.15 m
metric 6.02
metric 7.62
metric 4.00
metric 5.20 m
metric 13.46
metric 15.7
metric 13.59
metric 13.855
metric 50.75
metric 59.09
metric 50.02
metric 45.08
metric 59.76
metric 49.61
metric 5253
metric 5253
metric 66.2
metric 48.31
metric 1:49:39
metric 1:57:11
metric 39.36
metric 44.58
metric 3:08.82
metric 3:39.29
metric 11:00.85
metric 8:55.05


In [1306]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,11.75,100m,Female,11.750
1,10.443,100m,Male,10.443
2,21.02,200m,Male,21.020
3,23.6,200m,Female,23.600
4,46.63,400m,Male,46.630
5,53.84,400m,Female,53.840
6,2:09.15,800m,Female,129.150
7,1:53.86,800m,Male,113.860
8,4:26.33,1500m,Female,266.330
9,14:43.45,5000m,Male,883.450


In [1307]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [1308]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT'].str.strip()

In [1309]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2%,3.5%,5%,MAPPED_EVENT
0,11.75,100m,Female,11.750,11.98500,12.161250,12.33750,100m
1,10.443,100m,Male,10.443,10.65186,10.808505,10.96515,100m
2,21.02,200m,Male,21.020,21.44040,21.755700,22.07100,200m
3,23.6,200m,Female,23.600,24.07200,24.426000,24.78000,200m
4,46.63,400m,Male,46.630,47.56260,48.262050,48.96150,400m
5,53.84,400m,Female,53.840,54.91680,55.724400,56.53200,400m
6,2:09.15,800m,Female,129.150,131.73300,133.670250,135.60750,800m
7,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300,800m
8,4:26.33,1500m,Female,266.330,271.65660,275.651550,279.64650,1500m
9,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250,5000m


In [1310]:
athletes

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION,MAPPED_EVENT
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,None,Mid,Female,SEA Youth,2024,International,1500m
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International,Pole vault
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,None,Jump,Female,SEA Youth,2024,International,Pole vault
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International,2000m steeplechase
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,None,Steeplechase,Female,SEA Youth,2024,International,2000m steeplechase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43688,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43689,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43690,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,
43691,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,None,Sprint,Male,Pocari Sweat Run 2.4km 2024,2024,Local,


In [1311]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT,UNIQUE_ID,DOB,COUNTRY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,REGION,MAPPED_EVENT
474,Jun Jie Calvin Quek,None,51.8,Singapore,None,1,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Taiwan Athletics Open,2024,International,400m hurdles
494,Jun Jie Calvin Quek,None,51.79,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International,400m hurdles
501,Jun Jie Calvin Quek,None,53.19,Singapore,None,4,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,UAE Athletics Grand Prix,2024,International,400m hurdles
616,Jun Jie Calvin Quek,None,52.05,Singapore,None,8,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,11th Kinami Michitaka Memorial Athletics Meet,2024,International,400m hurdles
626,Jun Jie Calvin Quek,None,52.18,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles
1125,Jun Jie Calvin Quek,None,56.95,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,XXXIII Qosanov Memorial,2024,International,400m hurdles
1209,Jun Jie Calvin Quek,None,51.33,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,None,Hurdles,Male,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles


In [1312]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [1313]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [1314]:
df

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,COMPETITION,DATE,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,...,SEA Youth,2024,International,1500m,4:26.33,1500m,266.33,271.6566,275.65155,279.6465
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,...,SEA Youth,2024,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,...,SEA Youth,2024,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,...,SEA Youth,2024,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,...,SEA Youth,2024,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN


In [1315]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,COMPETITION,DATE,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
93,Jun Jie Calvin Quek,None,51.8,Singapore,None,1,400m Hurdles,None,26-Feb-96,nan,...,Taiwan Athletics Open,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
112,Jun Jie Calvin Quek,None,51.79,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
118,Jun Jie Calvin Quek,None,53.19,Singapore,None,4,400m Hurdles,None,26-Feb-96,nan,...,UAE Athletics Grand Prix,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
222,Jun Jie Calvin Quek,None,52.05,Singapore,None,8,400m Hurdles,None,26-Feb-96,nan,...,11th Kinami Michitaka Memorial Athletics Meet,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
230,Jun Jie Calvin Quek,None,52.18,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
311,Jun Jie Calvin Quek,None,56.95,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
351,Jun Jie Calvin Quek,None,51.33,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875


In [1316]:
df[df['MAPPED_EVENT']=='Discus throw']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,COMPETITION,DATE,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
861,Mo Thant Cin,None,23.89,NJC,None,5.0,Discus (1.00kg),None,None,None,...,NSG,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
862,Thang Ying Yue,None,28.42,HCI,None,3.0,Discus (1.00kg),None,None,None,...,NSG,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
863,Emma Tan Yi Min,None,31.98,RI,None,2.0,Discus (1.00kg),None,None,None,...,NSG,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
864,Neo Ee Nin,None,34.73,EJC,None,1.0,Discus (1.00kg),None,None,None,...,NSG,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
865,Feng Han Qi,None,27.27,HCI,None,4.0,Discus (1.00kg),None,None,None,...,NSG,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15151,"Tay, Kimmy Yu Tong",25.97m,22.50m,Singapore Sports School,15.0,6,Women Discus Throw (1kg) Open,K060Z09,2009-02-01 00:00:00.000,None,...,54th SA Inter Club Championships 2024,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
15152,"Chan, Cara",19.59m,24.25m,Ngee Ann Polytechnic,17.0,3,Women Discus Throw (1kg) Open,C449F07,2007-01-21 00:00:00.000,None,...,54th SA Inter Club Championships 2024,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
15153,"Foo, Qi Xin",33.57m,32.24m,Hwa Chong Alumni Association,23.0,1,Women Discus Throw (1kg) Open,Q897B01,2001-02-26 00:00:00.000,None,...,54th SA Inter Club Championships 2024,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826
15154,"Yew, Meredith",None,17.13m,Singapore Polytechnic,18.0,12,Women Discus Throw (1kg) Open,M434F06,2006-09-28 00:00:00.000,None,...,54th SA Inter Club Championships 2024,2024,Local,Discus throw,45.08,Discus throw,45.08,44.1784,43.5022,42.826


In [1317]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [1318]:
df

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,COMPETITION,DATE,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,None,2,1500m,None,2006,SGP,...,SEA Youth,2024,International,1500m,4:26.33,1500m,266.33,271.6566,275.65155,279.6465
1,ONG ASHLEE YUXI,None,3.1,Singapore,None,2,Pole vault,None,2006,SGP,...,SEA Youth,2024,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,None,5,Pole vault,None,2006,SGP,...,SEA Youth,2024,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,None,4,2000m steeplechase,None,2006,SGP,...,SEA Youth,2024,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,None,5,2000m steeplechase,None,2006,SGP,...,SEA Youth,2024,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,Pocari Sweat Run 2.4km 2024,2024,Local,,NaN,NaN,NaN,NaN,NaN,NaN


In [1319]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,COMPETITION,DATE,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%
93,Jun Jie Calvin Quek,None,51.8,Singapore,None,1,400m Hurdles,None,26-Feb-96,nan,...,Taiwan Athletics Open,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
112,Jun Jie Calvin Quek,None,51.79,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
118,Jun Jie Calvin Quek,None,53.19,Singapore,None,4,400m Hurdles,None,26-Feb-96,nan,...,UAE Athletics Grand Prix,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
222,Jun Jie Calvin Quek,None,52.05,Singapore,None,8,400m Hurdles,None,26-Feb-96,nan,...,11th Kinami Michitaka Memorial Athletics Meet,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
230,Jun Jie Calvin Quek,None,52.18,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
311,Jun Jie Calvin Quek,None,56.95,Singapore,None,3,400m Hurdles,None,26-Feb-96,nan,...,XXXIII Qosanov Memorial,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875
351,Jun Jie Calvin Quek,None,51.33,Singapore,None,2,400m Hurdles,None,26-Feb-96,nan,...,Second Belt and Road Invitational Athletics Me...,2024,International,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875


In [1320]:
df.to_csv('df.csv', sep=',', encoding='utf-8-sig', index=False)


In [1321]:
# Convert results and seed into seconds format

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,6]    # event description
    
    metric=df.iloc[rowIndex,2] # result
    metric_seed=df.iloc[rowIndex, 1]  #seed
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric==' DNS' or metric=='NH':
        continue
    
    result_out = convert_time(i, input_string, metric)
    print('line', i, input_string, metric, result_out)

    
    seed_out = convert_time(i, input_string, metric_seed)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out
    df.loc[rowIndex, 'SEED_CONV'] = seed_out


metric 04:55.0
line 0 1500m 04:55.0 295.0
metric None
metric 3.1
line 1 Pole vault 3.1 3.1
metric None
metric 2.3
line 2 Pole vault 2.3 2.3
metric None
metric 08:25.2
line 3 2000m steeplechase 08:25.2 505.2
metric None
metric 08:32.2
line 4 2000m steeplechase 08:32.2 512.2
metric None
metric 12.767
line 5 100m 12.767 12.767
metric None
metric 12.59
line 6 100m 12.59 12.59
metric None
metric 12.71
line 7 100m 12.71 12.71
metric None
metric 12.95
line 8 100m 12.95 12.95
metric None
metric 26.76
line 9 200m 26.76 26.76
metric None
metric 1.55
line 10 High jump 1.55 1.55
metric None
metric 34.54
line 11 Discus throw 34.54 34.54
metric None
metric 5.45
line 12 Long jump 5.45 5.45
metric None
metric 27.15
line 13 200m 27.15 27.15
metric None
metric 04:01.5
line 14 1500m 04:01.5 241.5
metric None
metric 1:53:51
line 15 800m 1:53:51 6831.0
metric None
metric 12.63w
W 12.63w
line 16 100m 12.63w 
metric None
metric 31:00:00
line 17 10km, Road 31:00:00 111600.0
metric None
metric 11.82
line 18 10

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/390498432.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/390498432.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'SEED_CONV'] = seed_out


metric 00:26.9
line 1224 200m 00:26.9 26.9
metric None
metric 00:24.6
line 1225 200m 00:24.6 24.6
metric None
metric 00:33.9
line 1226 200m 00:33.9 33.9
metric None
metric 00:30.2
line 1227 200m 00:30.2 30.2
metric None
metric 00:24.2
line 1228 200m 00:24.2 24.2
metric None
metric 00:25.5
line 1229 200m 00:25.5 25.5
metric None
metric 00:26.8
line 1230 200m 00:26.8 26.8
metric None
metric 01:06.5
line 1231 400m 01:06.5 66.5
metric None
metric 01:14.8
line 1232 400m 01:14.8 74.8
metric None
metric 01:03.1
line 1233 400m 01:03.1 63.1
metric None
metric 00:54.8
line 1234 400m 00:54.8 54.8
metric None
metric 01:13.8
line 1235 400m 01:13.8 73.8
metric None
metric 00:57.7
line 1236 400m 00:57.7 57.7
metric None
metric 01:08.1
line 1237 400m 01:08.1 68.1
metric None
metric 00:56.3
line 1238 400m 00:56.3 56.3
metric None
metric 01:15.9
line 1239 400m 01:15.9 75.9
metric None
metric 01:12.8
line 1240 400m 01:12.8 72.8
metric None
metric 01:15.3
line 1241 400m 01:15.3 75.3
metric None
metric 00:

metric 04:06.8
line 2554 4x400m Relay 04:06.8 246.8
metric None
metric 04:29.6
line 2555 4x400m Relay 04:29.6 269.6
metric None
metric 03:36.0
line 2556 4x400m Relay 03:36.0 216.0
metric None
metric 05:16.1
line 2557 4x400m Relay 05:16.1 316.1
metric None
metric 05:15.0
line 2558 4x400m Relay 05:15.0 315.0
metric None
metric 04:50.8
line 2559 4x400m Relay 04:50.8 290.8
metric None
metric 04:37.4
line 2560 4x400m Relay 04:37.4 277.4
metric None
metric 04:12.7
line 2561 4x400m Relay 04:12.7 252.7
metric None
metric 05:09.4
line 2562 4x400m Relay 05:09.4 309.4
metric None
metric 04:05.9
line 2563 4x400m Relay 04:05.9 245.9
metric None
metric 03:54.9
line 2564 4x400m Relay 03:54.9 234.9
metric None
metric 00:16.2
line 2565 100m Hurdles (0.762m) 00:16.2 16.2
metric None
metric 00:18.1
line 2566 100m Hurdles (0.762m) 00:18.1 18.1
metric None
metric 00:18.6
line 2567 100m Hurdles (0.762m) 00:18.6 18.6
metric None
metric 00:19.1
line 2568 100m Hurdles (0.762m) 00:19.1 19.1
metric None
metric 0

metric 00:57.0
line 3852 400m 00:57.0 57.0
metric None
metric 01:06.3
line 3853 400m 01:06.3 66.3
metric None
metric 03:47.9
line 3854 800m 03:47.9 227.9
metric None
metric 03:03.9
line 3855 800m 03:03.9 183.9
metric None
metric 03:03.6
line 3856 800m 03:03.6 183.6
metric None
metric 02:52.5
line 3857 800m 02:52.5 172.5
metric None
metric 03:12.8
line 3858 800m 03:12.8 192.8
metric None
metric 02:53.9
line 3859 800m 02:53.9 173.9
metric None
metric 02:21.8
line 3860 800m 02:21.8 141.8
metric None
metric 02:23.6
line 3861 800m 02:23.6 143.6
metric None
metric 03:21.2
line 3862 800m 03:21.2 201.2
metric None
metric 02:47.7
line 3863 800m 02:47.7 167.7
metric None
metric 03:02.3
line 3864 800m 03:02.3 182.3
metric None
metric 03:29.6
line 3865 800m 03:29.6 209.6
metric None
metric 02:48.3
line 3866 800m 02:48.3 168.3
metric None
metric 02:38.6
line 3867 800m 02:38.6 158.6
metric None
metric 02:37.1
line 3868 800m 02:37.1 157.1
metric None
metric 03:02.6
line 3869 800m 03:02.6 182.6
metric

...

metric 1:05.94
line 12972 Men 400 Meter Dash U20 1:05.94 65.94
metric 1:05.13
metric 56.07
line 12973 Men 400 Meter Dash U20 56.07 56.07
metric  
metric 53.39
line 12974 Men 400 Meter Dash U20 53.39 53.39
metric  
metric 55.61
line 12975 Men 400 Meter Dash U20 55.61 55.61
metric  
metric 14.15
line 12976 Women 100 Meter Dash U20 14.15 14.15
metric  
metric 14.33
line 12977 Women 100 Meter Dash U20 14.33 14.33
metric  
metric 14.68
line 12978 Women 100 Meter Dash U20 14.68 14.68
metric  
metric 13.12
line 12979 Women 100 Meter Dash U20 13.12 13.12
metric  
metric 15.73
line 12980 Women 100 Meter Dash U20 15.73 15.73
metric  
metric 13.74
line 12981 Women 100 Meter Dash U20 13.74 13.74
metric  
metric 14.82
line 12982 Women 100 Meter Dash U20 14.82 14.82
metric  
metric 28.51
line 12983 Women 200 Meter Dash U20 28.51 28.51
metric  
metric 29.87
line 12984 Women 200 Meter Dash U20 29.87 29.87
metric  
metric 30.01
line 12985 Women 200 Meter Dash U20 30.01 30.01
metric  
metric 31.50
line 

metric 51.25
metric 50.11
line 14266 Men 400 Meter Dash Open 50.11 50.11
metric 50.99
metric 53.34
line 14267 Men 400 Meter Dash Open 53.34 53.34
metric 52.16
metric 51.26
line 14268 Men 400 Meter Dash Open 51.26 51.26
metric 50.72
metric 47.98
line 14269 Men 400 Meter Dash Open 47.98 47.98
metric 47.97
metric 50.99
line 14270 Men 400 Meter Dash Open 50.99 50.99
metric 51.29
metric 50.04
line 14271 Men 400 Meter Dash Open 50.04 50.04
metric 50.99
metric 50.67
line 14272 Men 400 Meter Dash Open 50.67 50.67
metric 51.14
metric 49.82
line 14273 Men 400 Meter Dash Open 49.82 49.82
metric 51.14
metric 50.12
line 14274 Men 400 Meter Dash Open 50.12 50.12
metric 51.95
metric 51.13
line 14275 Men 400 Meter Dash Open 51.13 51.13
metric 51.29
metric 49.87
line 14276 Men 400 Meter Dash Open 49.87 49.87
metric 50.44
metric 50.23
line 14277 Men 400 Meter Dash Open 50.23 50.23
metric 51.95
metric 49.48
line 14278 Men 400 Meter Dash Open 49.48 49.48
metric 50.24
metric 16:36.06
line 14279 Men 5000 Me

metric 22.67
line 15580 Men 200 Meter Dash Open 22.67 22.67
metric 22.88
metric 22.26
line 15581 Men 200 Meter Dash Open 22.26 22.26
metric 21.80
metric 22.25
line 15582 Men 200 Meter Dash Open 22.25 22.25
metric 21.69
metric 23.39
line 15583 Men 200 Meter Dash Open 23.39 23.39
metric 23.00
metric 22.96
line 15584 Men 200 Meter Dash Open 22.96 22.96
metric 22.76
metric 23.72
line 15585 Men 200 Meter Dash Open 23.72 23.72
metric 23.20
metric 50.39
line 15586 Men 400 Meter Dash Open 50.39 50.39
metric 51.25
metric 50.44
line 15587 Men 400 Meter Dash Open 50.44 50.44
metric 50.44
metric 50.74
line 15588 Men 400 Meter Dash Open 50.74 50.74
metric 50.72
metric 52.17
line 15589 Men 400 Meter Dash Open 52.17 52.17
metric 52.67
metric 53.42
line 15590 Men 400 Meter Dash Open 53.42 53.42
metric 51.98
metric 51.21
line 15591 Men 400 Meter Dash Open 51.21 51.21
metric 51.99
metric 13.08
line 15592 Women 100 Meter Dash Open 13.08 13.08
metric 12.92
metric 12.00
line 15593 Women 100 Meter Dash Open

metric 15.28
line 16861 Girls 100 Meter Dash 11-12 15.28 15.28
metric None
metric 16.08
line 16862 Girls 100 Meter Dash 11-12 16.08 16.08
metric None
metric 16.30
line 16863 Girls 100 Meter Dash 11-12 16.30 16.3
metric None
metric 15.63
line 16864 Girls 100 Meter Dash 11-12 15.63 15.63
metric None
metric 31.58
line 16865 Girls 200 Meter Dash 11-12 31.58 31.58
metric None
metric 34.59
line 16866 Girls 200 Meter Dash 11-12 34.59 34.59
metric None
metric 31.96
line 16867 Girls 200 Meter Dash 11-12 31.96 31.96
metric None
metric 32.53
line 16868 Girls 200 Meter Dash 11-12 32.53 32.53
metric None
metric 31.81
line 16869 Girls 200 Meter Dash 11-12 31.81 31.81
metric None
metric 56.84
line 16870 Boys 4x100 Meter Relay 11-12 56.84 56.84
metric None
metric 57.76
line 16871 Boys 4x100 Meter Relay 11-12 57.76 57.76
metric None
metric 58.54
line 16872 Boys 4x100 Meter Relay 11-12 58.54 58.54
metric None
metric 1:04.56
line 16873 Boys 4x100 Meter Relay 11-12 1:04.56 64.56
metric None
metric 1:01.34

metric 58.40
line 18129 M40 400 Meter Dash Masters 58.40 58.4
metric None
metric 15.72
line 18130 W40 100 Meter Dash Masters 15.72 15.72
metric None
metric 14.49
line 18131 W40 100 Meter Dash Masters 14.49 14.49
metric None
metric 15.06
line 18132 W40 100 Meter Dash Masters 15.06 15.06
metric None
metric 16.90
line 18133 W40 100 Meter Dash Masters 16.90 16.9
metric None
metric 1:24.90
line 18134 W40 400 Meter Dash Masters 1:24.90 84.9
metric None
metric 15.32
line 18135 W45 100 Meter Dash Masters 15.32 15.32
metric None
metric 1:35.40
line 18136 W45 400 Meter Dash Masters 1:35.40 95.4
metric None
metric 1:21.90
line 18137 W45 400 Meter Dash Masters 1:21.90 81.9
metric None
metric 34.85
line 18138 W55 200 Meter Dash Masters 34.85 34.85
metric None
metric 37.48
line 18139 W55 200 Meter Dash Masters 37.48 37.48
metric None
metric 35.02
line 18140 W60 200 Meter Dash Masters 35.02 35.02
metric None
metric 38.56
line 18141 W65 200 Meter Dash Masters 38.56 38.56
metric None
metric 4:02.26
lin

In [1322]:
df["AGE"].fillna(0, inplace=True)
df['AGE'] = df['AGE'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_11111/2753303150.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AGE"].fillna(0, inplace=True)


In [1323]:
df

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,REGION,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,0.0,2,1500m,None,2006,SGP,...,International,1500m,4:26.33,1500m,266.33,271.6566,275.65155,279.6465,295.0,295.0
1,ONG ASHLEE YUXI,None,3.1,Singapore,0.0,2,Pole vault,None,2006,SGP,...,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000,3.1,3.1
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,0.0,5,Pole vault,None,2006,SGP,...,International,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000,2.3,2.3
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,0.0,4,2000m steeplechase,None,2006,SGP,...,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN,505.2,505.2
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,0.0,5,2000m steeplechase,None,2006,SGP,...,International,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN,512.2,512.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23.0,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,434.96,454.0
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27.0,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,458.04,458.0
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22.0,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,462.07,471.0
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25.0,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,Local,,NaN,NaN,NaN,NaN,NaN,NaN,475.82,476.0


In [1324]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [1325]:
df

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,0.0,2,1500m,None,2006,SGP,...,1500m,4:26.33,1500m,266.33,271.6566,275.65155,279.6465,295.0,295.0,295.0
1,ONG ASHLEE YUXI,None,3.1,Singapore,0.0,2,Pole vault,None,2006,SGP,...,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000,3.1,3.1,3.1
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,0.0,5,Pole vault,None,2006,SGP,...,Pole vault,4.00,Pole vault,4.00,3.9200,3.86000,3.8000,2.3,2.3,2.3
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,0.0,4,2000m steeplechase,None,2006,SGP,...,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN,505.2,505.2,505.2
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,0.0,5,2000m steeplechase,None,2006,SGP,...,2000m steeplechase,NaN,NaN,NaN,NaN,NaN,NaN,512.2,512.2,512.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23.0,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,434.96,454.0,434.96
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27.0,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,458.04,458.0,458.04
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22.0,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,462.07,471.0,462.07
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25.0,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,475.82,476.0,475.82


In [1326]:
df[df['NAME']=='Jun Jie Calvin Quek']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,MAPPED_EVENT,BENCHMARK,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST
93,Jun Jie Calvin Quek,None,51.8,Singapore,0.0,1,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,51.8,51.8,51.8
112,Jun Jie Calvin Quek,None,51.79,Singapore,0.0,2,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,51.79,51.79,51.79
118,Jun Jie Calvin Quek,None,53.19,Singapore,0.0,4,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,53.19,53.19,53.19
222,Jun Jie Calvin Quek,None,52.05,Singapore,0.0,8,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,52.05,52.05,52.05
230,Jun Jie Calvin Quek,None,52.18,Singapore,0.0,3,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,52.18,52.18,52.18
311,Jun Jie Calvin Quek,None,56.95,Singapore,0.0,3,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,56.95,56.95,56.95
351,Jun Jie Calvin Quek,None,51.33,Singapore,0.0,2,400m Hurdles,None,26-Feb-96,nan,...,400m hurdles,50.75,400m hurdles,50.75,51.765,52.52625,53.2875,51.33,51.33,51.33


In [1327]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST']] = df[['2%', '3.5%', '5%', 'RESULT_BEST']].apply(pd.to_numeric)

In [1328]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='SEA Games']

In [1329]:
df.to_csv("df_test.csv", encoding="utf-8")

In [1330]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [1331]:
df.to_csv('df_test.csv', sep=',', encoding='utf-8-sig', index=False)


In [1332]:
df[df['NAME']=='LEE, VANESSA']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
14320,"LEE, VANESSA",5:18.63,4:51.73,SINGAPORE,26.0,3,Women 1500 Meter Run Open,None,1998-02-23 00:00:00.000,None,...,275.65155,279.6465,291.73,318.63,291.73,-20.0734,-16.07845,-12.0835,-25.40,-4.537041
14368,"LEE, VANESSA",12:03.41,11:09.16,SINGAPORE,26.0,1,Women 3000 Meter Steeplechase Open,None,1998-02-23 00:00:00.000,None,...,683.97975,693.8925,669.16,723.41,669.16,4.9070,14.81975,24.7325,-8.31,3.742529


In [1333]:
# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Norhisham, Jamie El Redha Ang')



df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')






df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')




df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')
df['NAME'] = df['NAME'].replace(regex=r'KRISHNAN, HARIHARAN S/O', value='S/O Krishna, Hariharan')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Norhisham, Jamie El Redha Ang')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo, Alexis')



df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')


df['NAME'] = df['NAME'].replace(regex=r'Shun Yi Audrey Koh', value='KOH SHUN YI AUDREY')

df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')


df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'., Hariharan', value='S/O Krishna, Hariharan')
df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='Chia, Shawn')

df['NAME'] = df['NAME'].replace(regex=r'S/O KrishnS/O Krishna, Hariharan', value='S/O Krishna, Hariharan')


df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')

df['NAME'] = df['NAME'].replace(regex=r'YEE, CHUN WAI, ERIC', value='Yee Chun Wai, Eric')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Chun Wai, Eric', value='Yee Chun Wai, Eric')

df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')

df['NAME'] = df['NAME'].replace(regex=r'ESTHER TAY SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Esther Tay', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther', value='TAY, ESTHER')


df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='Low, Jun Yu')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='Low, Jun Yu')

df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')

df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara Q', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'HSIN-WEN CLARA, Chua', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua, Clara')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')



df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'TAN, JONATHAN', value='Tan, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'JONATHAN TAN', value='Tan, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Eric', value='Yee Chun Wai, Eric')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHUN WAI, ERIC', value='Yee Chun Wai, Eric')
df['NAME'] = df['NAME'].replace(regex=r'Eric Yee Chun Wai', value='Yee Chun Wai, Eric')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'CHEW, JADE', value='Chew, Jade')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')

df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER', value='Tay, Esther')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='Yee Chun Wai, Eric')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey Q', value='Koh Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Gabriel Lee', value='Lee, Gabriel')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther SHEE WEI', value='Tay, Esther')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Wei Guan, Chong', value='Chong, Wei Guan')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')



df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')
df['NAME'] = df['NAME'].replace(regex=r'Chua, Clara Q', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')


df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')

df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')


df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
# Correct some of the known name variations
# Need to fix 2024 NSG names extracts (extracting Q/q with name)

df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')


df['NAME'] = df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'TAN, SHEE RU, ELIZABETH-A', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Elizabeth-Ann Tan', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shee Ru, Elizabeth-Ann', value='TAN, ELIZABETH-ANN')

df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'LOUIS, MARC BRIAN BRIAN', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Louis, Marc Brian Brian', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Marc Brian Louis', value='Louis, Marc Brian')
df['NAME'] = df['NAME'].replace(regex=r'Brian louis, Marc', value='Louis, Marc Brian')




df['NAME'] = df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan, Jun Jie')
df['NAME'] = df['NAME'].replace(regex=r'Tan Jun Jie', value='Tan, Jun Jie')

df['NAME'] = df['NAME'].replace(regex=r'Shanti Veronica Pereira', value='PEREIRA, VERONICA SHANTI')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, YEE LING, BERNICE', value='Liew, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'LIEW, BERNICE YEE LING', value='Liew, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Mark Lee', value='LEE, MARK REN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Mark', value='LEE, MARK REN')





df['NAME'] = df['NAME'].replace(regex=r'MUN, IVAN', value='Mun, Ivan')

df['NAME'] = df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')
df['NAME'] = df['NAME'].replace(regex=r'YAP, SEAN YEE', value='Yap, Sean Yee')
df['NAME'] = df['NAME'].replace(regex=r'KRISHNAN, HARIHARAN S/O', value='S/O Krishna, Hariharan')

df['NAME'] = df['NAME'].replace(regex=r'ALEXIS TEO PEI QI', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Alexis Teo Pei Qi', value='Teo Pei Qi, Alexis')
df['NAME'] = df['NAME'].replace(regex=r'Teo Pei Qi, Alexis Q', value='Teo Pei Qi, Alexis')




df['NAME'] = df['NAME'].replace(regex=r'Enriquez, Roxanne Rose Zulueta', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROXANNE ROSE ZULUETA', value='ROSE ZULUETA ENRIQUE, ROXANNE')
df['NAME'] = df['NAME'].replace(regex=r'ENRIQUEZ, ROSE ZULUETA, ROXANN', value='ROSE ZULUETA ENRIQUE, ROXANNE')

df['NAME'] = df['NAME'].replace(regex=r'Goh, Clara', value='GOH, CLARA')

df['NAME'] = df['NAME'].replace(regex=r'WONG JIN ROU', value='Wong, Jin Rou')


df['NAME'] = df['NAME'].replace(regex=r'Shun Yi Audrey Koh', value='KOH SHUN YI AUDREY')

df['NAME'] = df['NAME'].replace(regex=r'THANA RAJAN, S/O, THIRUBEN', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Rajan Thiruben Thana', value='THANA RAJAN, THIRUBEN  S/O')
df['NAME'] = df['NAME'].replace(regex=r'Thana Rajan, Thiruben', value='THANA RAJAN, THIRUBEN  S/O')



df['NAME'] = df['NAME'].replace(regex=r'Raam Kumar Muthukumaran', value='Muthukumaran, Raam Kumar')

df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='Chia, Shawn')
df['NAME'] = df['NAME'].replace(regex=r'., Hariharan', value='S/O Krishna, Hariharan')

df['NAME'] = df['NAME'].replace(regex=r'S/O KrishnS/O Krishna, Hariharan', value='S/O Krishna, Hariharan')


df['NAME'] = df['NAME'].replace(regex=r'QUEK XIN YONG JEREMIAH', value='QUEK, JEREMIAH')
df['NAME'] = df['NAME'].replace(regex=r'quek xin yong jeremi', value='QUEK, JEREMIAH')


df['NAME'] = df['NAME'].replace(regex=r'Chiu, Jingwen', value='Chiu, Jing Wen')



df['NAME'] = df['NAME'].replace(regex=r'LEE YU FOONG (LI YUFENG)', value='Lee, Yu Foong')


df['NAME'] = df['NAME'].replace(regex=r'LOW CHUAN YI', value='Low, Chuan Yi')

df['NAME'] = df['NAME'].replace(regex=r'ESTHER TAY SHEE WEI', value='Tay, Esther')
df['NAME'] = df['NAME'].replace(regex=r'Esther Tay', value='Tay, Esther')

df['NAME'] = df['NAME'].replace(regex=r'Rachel Yang Bing Jie', value='YANG, RACHEL')


df['NAME'] = df['NAME'].replace(regex=r'Dewey Ng', value='Ng, Dewey')
df['NAME'] = df['NAME'].replace(regex=r'NG, DEWEY', value='Ng, Dewey')

df['NAME'] = df['NAME'].replace(regex=r'., Brendon Ting Li King', value='Ting Li King, Brandon')
df['NAME'] = df['NAME'].replace(regex=r'TAN KIAN SWEE ISAAC', value='Tan, Isaac')
df['NAME'] = df['NAME'].replace(regex=r'TAN, KIAN SWEE ISAAC', value='Tan, Isaac')

df['NAME'] = df['NAME'].replace(regex=r'GAO SHENGWEI', value='Gao, Shengwei')
df['NAME'] = df['NAME'].replace(regex=r'NEO SHYH KAI', value='Neo, Shyh Kai')
df['NAME'] = df['NAME'].replace(regex=r'ANGEL LIM YU XIN', value='Lim, Yuxin Angel')
df['NAME'] = df['NAME'].replace(regex=r'LAUREL, LIM JIA EN', value='LAUREL LIM JIA EN')

df['NAME'] = df['NAME'].replace(regex=r'LOH DING RONG, ANSON', value='Loh, Anson')

df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')
df['NAME'] = df['NAME'].replace(regex=r'NG JUN HENG, RYAN', value='Ng, Ryan')
df['NAME'] = df['NAME'].replace(regex=r'LIM WEE ENG', value='Lim, Wee Eng')


df['NAME'] = df['NAME'].replace(regex=r'Chen Xiang Ang', value='Ang, Chen Xiang')

df['NAME'] = df['NAME'].replace(regex=r'CHUA JE-AN, GARRETT', value='Chua, Garrett')
df['NAME'] = df['NAME'].replace(regex=r'Garrett Chua', value='Chua, Garrett')


df['NAME'] = df['NAME'].replace(regex=r'Looi, Yan Ning Samantha', value='LOOI YAN NING, SAMANTHA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Sheila Qi Le', value='Lim Qi Le, Sheila')
df['NAME'] = df['NAME'].replace(regex=r'NG, CHIN HUI', value='Ng, Chin Hui')
df['NAME'] = df['NAME'].replace(regex=r'SEAN, TEE YU XIANG', value='SEAN TEE YU XIANG')

df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'Chua Hsin-Wen Clara Q', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'HSIN-WEN CLARA, Chua', value='Chua Hsin-Wen, Clara')
df['NAME'] = df['NAME'].replace(regex=r'CHUA HSIN-WEN CLARA', value='Chua Hsin-Wen, Clara')


df['NAME'] = df['NAME'].replace(regex=r'Edsel Poh', value='Poh, Edsel')
df['NAME'] = df['NAME'].replace(regex=r'Gautam, Kulandaiveludayar Se', value='Kulandaiveludayar Se, Gautam')

df['NAME'] = df['NAME'].replace(regex=r'FAITH ZHEN FORD', value='Faith Zhen Ford')
df['NAME'] = df['NAME'].replace(regex=r'Ford, Faith', value='Faith Zhen Ford')
df['NAME'] = df['NAME'].replace(regex=r'FORD, FAITH', value='Faith Zhen Ford')


df['NAME'] = df['NAME'].replace(regex=r'Hanim, Asmah', value='Asmah, Hanim')
df['NAME'] = df['NAME'].replace(regex=r'TAN, JONATHAN', value='Tan, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'JONATHAN TAN', value='Tan, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'YOON SAE YOUNG', value='Yoon, Sae Young')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Alexis Li Xuan', value='Wong, Alexis')

df['NAME'] = df['NAME'].replace(regex=r'TAN JUN ERN KEAN', value='Tan Jun Ern Kean')

df['NAME'] = df['NAME'].replace(regex=r'JOEY LIM ZI YI', value='Lim, Joey')
df['NAME'] = df['NAME'].replace(regex=r'Bernie Chong Yi Shan', value='Chong, Bernie')

df['NAME'] = df['NAME'].replace(regex=r'Tan Ting Jun', value='Tan, Ting Jun')
df['NAME'] = df['NAME'].replace(regex=r'Xiao, Hongyang', value='Xiao, Hong Yang')
df['NAME'] = df['NAME'].replace(regex=r'Reiner Leong Rong Xi', value='Leong, Reiner')


df['NAME'] = df['NAME'].replace(regex=r'CHEN JING YING JANELLE', value='Chen, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Toh Wei De', value='Toh, Wei De')
#df['NAME'] = df['NAME'].replace(regex=r'SheikAluu\'udin\,\sSkeik\sFayz', value='Sheik\sAluu\'udin\,\sSkeik\sFayz')

df['NAME'] = df['NAME'].replace(regex=r'kapil, Arnav', value='Kapil, Arnav')
df['NAME'] = df['NAME'].replace(regex=r'KIRAN, SEBASTIAN JEHUNATHAN', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'LAU, REINA', value='Lau, Reina')
df['NAME'] = df['NAME'].replace(regex=r'Lau Reina', value='Lau, Reina')


df['NAME'] = df['NAME'].replace(regex=r'Zou Aolin q', value='Zou Aolin')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'CHEW, JADE', value='Chew, Jade')

df['NAME'] = df['NAME'].replace(regex=r'NEO EE , NIN', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Neo Ee Nin', value='Neo, Ee Nin')
df['NAME'] = df['NAME'].replace(regex=r'Emma Tan Yi Min', value='TAN, EMMA YI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Emma Yi Min', value='TAN, EMMA YI MIN')


df['NAME'] = df['NAME'].replace(regex=r'Si Hui, Amanda, Loo', value='Amanda, Loo Si Hui')
df['NAME'] = df['NAME'].replace(regex=r'Si Hui Amanda, Loo', value='Amanda, Loo Si Hui')



df['NAME'] = df['NAME'].replace(regex=r'WONG, JIN ROU', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Seah, Michelle Xin Yi', value='Seah, Michelle')
df['NAME'] = df['NAME'].replace(regex=r'Jing Wen, Chiu', value='Chiu, Jing Wen')
df['NAME'] = df['NAME'].replace(regex=r'NG, ROY', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Kai Leong Roy', value='Ng Kai Leong, Roy')
df['NAME'] = df['NAME'].replace(regex=r'., Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ashlee Ong Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER', value='Tay, Esther')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'., Cheong Yin Yern Jose', value='Joseph Cheong Yin Yern')
df['NAME'] = df['NAME'].replace(regex=r'Thang Ying Yue', value='Thang, Ying Yue')
df['NAME'] = df['NAME'].replace(regex=r'Hongyang, Xiao', value='XIAO, HONGYANG')
df['NAME'] = df['NAME'].replace(regex=r'Tan Shou Yi Rei (Chen Shouyi)', value='Tan Shou Yi Rei')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Ashlee Yuxi Yuxi', value='Ong, Ashlee Yuxi')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jonathan', value='Low Jun Jie, Jonathan')
df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey Q', value='Koh Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Murali, Sham', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'SHANJIE, MURALI SHAM KUMAR', value='Kumar Shanjie, Murali Sham')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Aaron Xu Qing', value='Koh, Aaron')

df['NAME'] = df['NAME'].replace(regex=r'^Tan\sShou\sYi\sRei$', value='Tan Shou Yi Rei (Chen Shouyi)')



df['NAME'] = df['NAME'].replace(regex=r'KUMAR, VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'KUMAR, S VIRESH', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r's Viresh Kumar', value='Kumar, Viresh')
df['NAME'] = df['NAME'].replace(regex=r'Kumar, S Viresh', value='Kumar, Viresh')


df['NAME'] = df['NAME'].replace(regex=r'Mook, He Jun', value='HE, MOOK JUN')
df['NAME'] = df['NAME'].replace(regex=r'Gabriel Lee', value='Lee, Gabriel')
df['NAME'] = df['NAME'].replace(regex=r'Leong, Lorraine', value='Leong Jia En, Lorraine')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther SHEE WEI', value='Tay, Esther')
df['NAME'] = df['NAME'].replace(regex=r'Joseph Cheong Yin Yern', value='Cheong, Joseph')
df['NAME'] = df['NAME'].replace(regex=r'Wee, Junn Elliott', value='Wee, Elliot')
df['NAME'] = df['NAME'].replace(regex=r'Sheik, Fayz', value='Sheik Alau\'ddin, Sheik Fayz')
df['NAME'] = df['NAME'].replace(regex=r'Wei Guan, Chong', value='Chong, Wei Guan')
df['NAME'] = df['NAME'].replace(regex=r'Keondre Liew Zhen Kai', value='Liew, Keondre Zhen Kai')




df['NAME'] = df['NAME'].replace(regex=r'TOH, TEDD', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Toh, Tedd', value='Toh Jun Xi, Tedd')





df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='ZHONG, CHUHAN')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Daniel', value='Lee Zheng Xian, Daniel Christopher')

df['NAME'] = df['NAME'].replace(regex=r'Chua, Clara Q', value='Chua, Clara')
df['NAME'] = df['NAME'].replace(regex=r'CHUA, HSIN-WEN CLARA', value='Chua, Clara')


df['NAME'] = df['NAME'].replace(regex=r'GOH, YEN YOUNG, AMELIA', value='Goh Yen Young Amelia')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Liew, Keondre Zhen Kai', value='LIEW, KEONDRE')
df['NAME'] = df['NAME'].replace(regex=r'Janelle, Lum', value='Lum Wai Yan, Janelle')
df['NAME'] = df['NAME'].replace(regex=r'Teh Ying Shan', value='Teh, Ying Shan')
df['NAME'] = df['NAME'].replace(regex=r'Ying Shan, Teh', value='Teh, Ying Shan')


df['NAME'] = df['NAME'].replace(regex=r'Mayah Elizabeth Kaufmann Q', value='Mayah Elizabeth Kaufmann')
df['NAME'] = df['NAME'].replace(regex=r'TNG, KAI XIN', value='Tng Kai Xin')
df['NAME'] = df['NAME'].replace(regex=r'YANG, BOBBI', value='Bobbi Victoria Yang Yen (Weng Xuan)')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Si En Tabitha', value='Ng Si En, Tabitha')
df['NAME'] = df['NAME'].replace(regex=r'Hoh Ying Min', value='Hoh, Ying Min')
df['NAME'] = df['NAME'].replace(regex=r'Tedd Toh', value='Toh Jun Xi, Tedd')
df['NAME'] = df['NAME'].replace(regex=r'Tung Hon Andrew Pak', value='Pak, Andrew')
df['NAME'] = df['NAME'].replace(regex=r'Irfan Qabeel Mohamman', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'Sebastian, Kiran', value='Sebastian Jehunathan, Kiran')
df['NAME'] = df['NAME'].replace(regex=r'TANG, CAYMAN', value='Tang Kai Sheng, Cayman')
df['NAME'] = df['NAME'].replace(regex=r'ZHONG, CHUHAN', value='Zhong, Chuhan')

df['NAME'] = df['NAME'].replace(regex=r'Reuben Lee Siong En', value='LEE SIONG EN, REUBEN RAINER')
df['NAME'] = df['NAME'].replace(regex=r'Tam, Joash', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Pooran Sankaresh Dharmaraj', value='Pooran, Dharmaraj')

df['NAME'] = df['NAME'].replace(regex=r'Meena, Girish Chandramohan', value='Chandramohan Meena, Girish')
df['NAME'] = df['NAME'].replace(regex=r'Wong Jin Rou', value='Wong, Jin Rou')
df['NAME'] = df['NAME'].replace(regex=r'Jayden Tan', value='Tan, Jayden')
df['NAME'] = df['NAME'].replace(regex=r'MOHHAMMAD IRFAN, QABEEL', value='Mohammad Irfan, Qabeel')
df['NAME'] = df['NAME'].replace(regex=r'BIN MD, MOHAMMAD IRFAN QABEE', value='Mohammad Irfan, Qabeel')



df['NAME'] = df['NAME'].replace(regex=r'TAM, JOASH', value='Tam Jong-Hng, Joash')
df['NAME'] = df['NAME'].replace(regex=r'Verboon Lara', value='Verboon, Lara')
df['NAME'] = df['NAME'].replace(regex=r'Verboon , Lara', value='Verboon, Lara')

df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Megan Anne Ying Ka Mun')
df['NAME'] = df['NAME'].replace(regex=r'Woong, Hui Min Rachel', value='Woong, Rachel Hui Min')

df['NAME'] = df['NAME'].replace(regex=r'FENG, HAN LIN', value='Feng, Han Lin')
df['NAME'] = df['NAME'].replace(regex=r'Angela Tan Shi Jin', value='TAN, ANGELA')
df['NAME'] = df['NAME'].replace(regex=r'Zhang, Ruiheng, Zayden', value='Zhang, Zayden')
df['NAME'] = df['NAME'].replace(regex=r'Jing Heng Ng', value='Ng, Jing Heng')



df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie El Redha Ang El Redha Ang', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Jamie Norhisham', value='Jamie El-Red Ang')
df['NAME'] = df['NAME'].replace(regex=r'Norhisham, Jamie', value='Jamie El-Red Ang')

### 2024###

df['NAME'] = df['NAME'].replace(regex=r'D/O JAIGANTH, LAAVINIA', value='Jaiganth, Laavinia')
df['NAME'] = df['NAME'].replace(regex=r'Jaiganth, Laavinia', value='JAIGANTH, LAAVINIA')


df['NAME'] = df['NAME'].replace(regex=r'Faith Zhen Ford', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford, Faith', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'FORD, FAITH', value='FAITH ZHEN FORD')
df['NAME'] = df['NAME'].replace(regex=r'Ford Faith', value='FAITH ZHEN FORD')

df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey', value='Koh, Shun Yi Audrey')
df['NAME'] = df['NAME'].replace(regex=r'Koh Shun Yi Audrey Q', value='Koh, Shun Yi Audrey')

df['NAME'] = df['NAME'].replace(regex=r'Romaine Soh', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Romaine', value='ROMAINE SOH RUI MIN')
df['NAME'] = df['NAME'].replace(regex=r'Soh, Rui Min Romaine', value='ROMAINE SOH RUI MIN')

df['NAME'] = df['NAME'].replace(regex=r'khoo, Tyeisha rene misson', value='Khoo, Tyeisha')

df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh Q', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'FAITH JIA YIN KOH', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Faith Jia Yin Koh', value='Koh, Faith Jia Yin')
df['NAME'] = df['NAME'].replace(regex=r'Koh, Faith', value='Koh, Faith Jia Yin')

df['NAME'] = df['NAME'].replace(regex=r'LIM, EIFFER YING QI', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'lim, eiffer', value='LIM YING QI, EIFFER')
df['NAME'] = df['NAME'].replace(regex=r'Eiffer, Lim Ying Qi', value='LIM YING QI, EIFFER')

df['NAME'] = df['NAME'].replace(regex=r'Chui Ling Goh', value='GOH, CHUI LING')

df['NAME'] = df['NAME'].replace(regex=r'Chong Le Xuan, Chloe', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, LE XUAN, CHLOE', value='CHONG LE XUAN, CHLOE')
df['NAME'] = df['NAME'].replace(regex=r'CHONG, CHLOE LE XUAN', value='CHONG LE XUAN, CHLOE')



df['NAME'] = df['NAME'].replace(regex=r'SIDHOM, MANUELA', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom Q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom q', value='Sidhom, Manuela')
df['NAME'] = df['NAME'].replace(regex=r'Manuela Sidhom', value='Sidhom, Manuela')

df['NAME'] = df['NAME'].replace(regex=r'^Gan,\sSteffi$', value='Gan, Steffi Kai Lin')
df['NAME'] = df['NAME'].replace(regex=r'Gan Kai-lin, Steffi', value='Gan, Steffi Kai Lin')

df['NAME'] = df['NAME'].replace(regex=r'Shing Ling Goh', value='GOH SHING LING')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shing Ling', value='GOH SHING LING')



df['NAME'] = df['NAME'].replace(regex=r'Yu En Bernice, Tan', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Yu En Bernice', value='Tan Yu En, Bernice')
#df['NAME'] = df['NAME'].replace(regex=r'Tan, Bernice', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice q', value='Tan Yu En, Bernice')
df['NAME'] = df['NAME'].replace(regex=r'Tan Yu En Bernice Q', value='Tan Yu En, Bernice')
#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2010-10-05')), 'Tan Yu En, Bernice', 'Tan, Bernice')



df['NAME'] = df['NAME'].replace(regex=r'TAN, YUN RU BERNICE', value='Tan Yun Ru, Bernice')

#df['NAME'] = np.where(((df['NAME']=='Tan, Bernice')&(df['DOB']=='2000-11-16')), 'Tan Yun Ru, Bernice', 'Tan, Bernice')

df['NAME'] = df['NAME'].replace(regex=r'Ng, Yew Cheo', value='NG, YEW CHEO')

df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling Q', value='Chew Kai Ling, Caelyn')
df['NAME'] = df['NAME'].replace(regex=r'Caelyn Chew Kai Ling', value='Chew Kai Ling, Caelyn')

df['NAME'] = df['NAME'].replace(regex=r'FOO, NATALIE ZHI NING', value='Foo Zhi Ning, Natalie')
df['NAME'] = df['NAME'].replace(regex=r'Foo, Natalie', value='Foo Zhi Ning, Natalie')


df['NAME'] = df['NAME'].replace(regex=r'Xiu Yi, Leong', value='Leong, Xiu Yi')


df['NAME'] = df['NAME'].replace(regex=r'Shing Ling Goh', value='Goh Shing Ling')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shing Ling', value='Goh Shing Ling')

df['NAME'] = df['NAME'].replace(regex=r'Tia Louise Rozario', value='ROZARIO, TIA LOUISE')
df['NAME'] = df['NAME'].replace(regex=r'Rozario, Tia Louise', value='ROZARIO, TIA LOUISE')


df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chloe Chee En-Ya Q', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'En-ya, Chloe', value='CHLOE CHEE EN-YA')
df['NAME'] = df['NAME'].replace(regex=r'Chee En-ya, Chloe', value='CHLOE CHEE EN-YA')


df['NAME'] = df['NAME'].replace(regex=r'Chuhan Zhong', value='Zhong, Chuhan')
df['NAME'] = df['NAME'].replace(regex=r'Megan Anne Ying Ka Mun Q', value='Zhong,')


df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara Q', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'LIM, YEE CHERN CLARA', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Clara Lim Yee Chern', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'Lim Yee Chern Clara', value='LIM YEE CHERN CLARA')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHERN CLARA LIM', value='LIM YEE CHERN CLARA')


df['NAME'] = df['NAME'].replace(regex=r'CHEW, JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Jade, Chew Yiu Tse', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Jade', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'CHEW, YIU TSE JADE', value='CHEW YIU TSE JADE')
df['NAME'] = df['NAME'].replace(regex=r'Chew, Yiu Tse Jade', value='CHEW YIU TSE JADE')


df['NAME'] = df['NAME'].replace(regex=r'Esther Tay', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER SHEE WEI', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'TAY, ESTHER', value='TAY, ESTHER')
df['NAME'] = df['NAME'].replace(regex=r'Tay, Esther', value='TAY, ESTHER')

df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'FENG HAN QI', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Han Qi Feng', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng Han Qi', value='FENG, HAN QI')
df['NAME'] = df['NAME'].replace(regex=r'Feng, Han Qi', value='FENG, HAN QI')

df['NAME'] = df['NAME'].replace(regex=r'Syed Ahmed Riadh, Sharifah Falisha', value='SYED AHMED RIADH, SHARIFAH FALISHA')
df['NAME'] = df['NAME'].replace(regex=r'Sharifah Falisha Binte Syed Ahmed', value='SYED AHMED RIADH, SHARIFAH FALISHA')

df['NAME'] = df['NAME'].replace(regex=r'Sam, Vanessa', value='SAM, VANESSA')

df['NAME'] = df['NAME'].replace(regex=r'Vanessa Lee Ying Zhuang', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee Ying Zhuang, Vanessa', value='LEE, VANESSA')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Vanessa', value='LEE, VANESSA')


df['NAME'] = df['NAME'].replace(regex=r'SNG, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Sng, Michelle', value='MICHELLE SNG')
df['NAME'] = df['NAME'].replace(regex=r'Michelle Sng Suat Li', value='MICHELLE SNG')

df['NAME'] = df['NAME'].replace(regex=r'Rachel See Boon Lay', value='Rachel See')

df['NAME'] = df['NAME'].replace(regex=r'Yee, Wai Teng Melissa', value='YEE, MELISSA')
df['NAME'] = df['NAME'].replace(regex=r'YEE WAI TENG, MELISSA', value='YEE, MELISSA')

df['NAME'] = df['NAME'].replace(regex=r'Kerstin Ong Jing Rong', value='ONG, KERSTIN')
df['NAME'] = df['NAME'].replace(regex=r'Ong, Kerstin', value='ONG, KERSTIN')


df['NAME'] = df['NAME'].replace(regex=r'Chua, Joshua', value='CHUA, JOSHUA')

df['NAME'] = df['NAME'].replace(regex=r'Raphael, Ryan', value='RAPHAEL, RYAN')
df['NAME'] = df['NAME'].replace(regex=r'NG, RYAN RAPHAEL', value='RAPHAEL, RYAN')

df['NAME'] = df['NAME'].replace(regex=r'Xander Ho Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho, Xander Ann Heng', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'HO, XANDER, ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'XANDER, HO ANN HENG', value='HO ANN HENG, XANDER')
df['NAME'] = df['NAME'].replace(regex=r'Ho Ann Heng, Xander', value='HO ANN HENG, XANDER')

df['NAME'] = df['NAME'].replace(regex=r'Tan, Tate', value='TATE TAN FUNG')

df['NAME'] = df['NAME'].replace(regex=r'Tan Hong An, Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan Daryl', value='TAN, DARYL')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Daryl', value='TAN, DARYL')

df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat', value='ONG, YING TAT')
df['NAME'] = df['NAME'].replace(regex=r'Ong Ying Tat Q', value='ONG, YING TAT')

df['NAME'] = df['NAME'].replace(regex=r'S/O SUBASH SOMAN, PRAHARSH RYAN', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan, Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'Ryan Praharsh', value='RYAN, PRAHARSH')
df['NAME'] = df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='RYAN, PRAHARSH')

df['NAME'] = df['NAME'].replace(regex=r'LIM, CHUN KHAI, JON', value='LIM, JON')
df['NAME'] = df['NAME'].replace(regex=r'Lim, Jon', value='LIM, JON')

df['NAME'] = df['NAME'].replace(regex=r'Wei Guan, Chong', value='CHONG, WEI GUAN')
df['NAME'] = df['NAME'].replace(regex=r'Chong, Wei Guan', value='CHONG, WEI GUAN')

df['NAME'] = df['NAME'].replace(regex=r'Mohamed, Haja Fayiz', value='FAYIZ, MOHAMED HAJA')
df['NAME'] = df['NAME'].replace(regex=r'Osman, Amir Rusyaidi', value='OSMAN, AMIR RUSYAIDI')

df['NAME'] = df['NAME'].replace(regex=r'Jing Heng Ng', value='NG, JING HENG')
df['NAME'] = df['NAME'].replace(regex=r'Ng, Jing Heng', value='NG, JING HENG')

df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan', value='TAN, JONATHAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jonathan Tan', value='TAN, JONATHAN')

df['NAME'] = df['NAME'].replace(regex=r'Lucas Le Cong Fun', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun Le Cong, Lucas Q', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Fun, Le Cong Lucas', value='FUN LE CONG LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'FUN, LE CONG LUCAS', value='FUN LE CONG LUCAS')


df['NAME'] = df['NAME'].replace(regex=r'PERCY MUNCHERJI, ZUBIN', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Muncherji, Zubin', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'MUNCHERJI, ZUBIN PERCY', value='ZUBIN PERCY MUNCHERJI')
df['NAME'] = df['NAME'].replace(regex=r'Zubin Percy Muncherji', value='ZUBIN PERCY MUNCHERJI')

df['NAME'] = df['NAME'].replace(regex=r'Quek, Jun Jie Calvin', value='QUEK, JUN JIE CALVIN')
df['NAME'] = df['NAME'].replace(regex=r'Jun Jie Calvin Quek', value='QUEK, JUN JIE CALVIN')

df['NAME'] = df['NAME'].replace(regex=r'Chia, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, WEI EN, SHAWN', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Chia Wei En, Shawn', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Wei En Shawn, Chia', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'Shawn Chia Wei En', value='SHAWN CHIA WEI EN')
df['NAME'] = df['NAME'].replace(regex=r'CHIA, SHAWN WEI EN', value='SHAWN CHIA WEI EN')


df['NAME'] = df['NAME'].replace(regex=r'Jie Cong Jayden, Tan', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'TAN , JIE CONG, JAYDEN', value='JIE CONG JAYDEN, TAN')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Jayden', value='JIE CONG JAYDEN, TAN')

df['NAME'] = df['NAME'].replace(regex=r'Lim, Oliver', value='LIM, OLIVER')

df['NAME'] = df['NAME'].replace(regex=r'YEE, ERIC CHUN WAI', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'YEE CHUN WAI, ERIC', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Eric Yee Chun Wai', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee, Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')
df['NAME'] = df['NAME'].replace(regex=r'Yee Chun Wai, Eric', value='YEE, CHUN WAI, ERIC')

df['NAME'] = df['NAME'].replace(regex=r'Rui Yong Soh', value='SOH RUI YONG GUILLAUME')

df['NAME'] = df['NAME'].replace(regex=r'TAN, AARON JUSTIN', value='TAN, AARON JUSTIN WEN JIE')
df['NAME'] = df['NAME'].replace(regex=r'Tan, Aaron Justin', value='TAN, AARON JUSTIN WEN JIE')


df['NAME'] = df['NAME'].replace(regex=r'Shaun, Goh', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Goh, Shaun', value='GOH, SHAUN')
df['NAME'] = df['NAME'].replace(regex=r'Shaun Goh', value='GOH, SHAUN')

df['NAME'] = df['NAME'].replace(regex=r'Ethan Yan', value='YAN, ETHAN')

df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Heng, Brandon', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'Brandon Heng Fu Hong', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG, BRANDON', value='HENG, BRANDON')
df['NAME'] = df['NAME'].replace(regex=r'HENG, FU HONG BRANDON', value='HENG, BRANDON')

df['NAME'] = df['NAME'].replace(regex=r'Tung Hon Andrew Pak', value='PAK, ANDREW')
df['NAME'] = df['NAME'].replace(regex=r'Pak, Andrew', value='PAK, ANDREW')

df['NAME'] = df['NAME'].replace(regex=r'Wong Yijie Lucas', value='WONG YIJIE LUCAS')
df['NAME'] = df['NAME'].replace(regex=r'Wong, Yijie Lucas', value='WONG YIJIE LUCAS')

df['NAME'] = df['NAME'].replace(regex=r'Mun Jern Wei', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Jern Wei Ivan', value='MUN, IVAN')
df['NAME'] = df['NAME'].replace(regex=r'Mun, Ivan', value='MUN, IVAN')


df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad, Kangli Emery', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Kangli Emery Conrad', value='EMERY, CONRAD')
df['NAME'] = df['NAME'].replace(regex=r'Conrad Kangli Emery Q', value='EMERY, CONRAD')

df['NAME'] = df['NAME'].replace(regex=r'Melvin, Wong', value='WONG, MELVIN')

df['NAME'] = df['NAME'].replace(regex=r'Caleb Hia', value='HIA, CALEB')

df['NAME'] = df['NAME'].replace(regex=r'Jun Yu Low', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low Jun Yu', value='LOW, JUN YU')
df['NAME'] = df['NAME'].replace(regex=r'Low, Jun Yu', value='LOW, JUN YU')

df['NAME'] = df['NAME'].replace(regex=r'Gabriel Lee', value='LEE GABRIEL JIN YI')
df['NAME'] = df['NAME'].replace(regex=r'Lee, Gabriel', value='LEE GABRIEL JIN YI')

df['NAME'] = df['NAME'].replace(regex=r'Andrew George Medina', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'Medina, Andrew George', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George Medina, Andrew', value='MEDINA, ANDREW GEORGE')
df['NAME'] = df['NAME'].replace(regex=r'George George Medina, Andrew', value='MEDINA, ANDREW GEORGE')



df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfann', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'^Curran,\sHarry$', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran Q', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Harry Irfan Curran', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'Curran, Harry Irfan', value='HARRY IRFAN CURRAN')
df['NAME'] = df['NAME'].replace(regex=r'CURRAN, HARRY IRFAN', value='HARRY IRFAN CURRAN')


df['NAME'] = df['NAME'].replace(regex=r'Wei Jun Huang', value='HUANG WEI JUN')
df['NAME'] = df['NAME'].replace(regex=r'HUANG, WEI JUN', value='HUANG WEI JUN')


In [1356]:
df[df['NAME']=='CHEW YIU TSE JADE']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
13691,CHEW YIU TSE JADE,,5.26m,Hwa Chong Alumni Association,20.0,3,Women Long Jump Open,None,2004-01-14,None,...,5.80930,5.7190,5.26,5.26,5.26,-0.6396,-0.54930,-0.4590,-0.76,-7.624585
13818,CHEW YIU TSE JADE,None,5.02m,Singapore,20.0,1,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.02,5.02,5.02,-0.8796,-0.78930,-0.6990,-1.00,-11.611296
14183,CHEW YIU TSE JADE,1.55m,1.60m,SINGAPORE,20.0,2,Women High Jump Open,None,2004-01-14 00:00:00.000,None,...,1.66945,1.6435,1.6,1.55,1.60,-0.0954,-0.06945,-0.0435,-0.13,-2.514451
14192,CHEW YIU TSE JADE,5.26m,5.29m,SINGAPORE,20.0,3,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.29,5.26,5.29,-0.6096,-0.51930,-0.4290,-0.73,-7.126246
14427,CHEW YIU TSE JADE,,1.55m,National University Singapore,0.0,1,Women High Jump Open,None,None,None,...,1.66945,1.6435,1.55,1.55,1.55,-0.1454,-0.11945,-0.0935,-0.18,-5.404624
14834,CHEW YIU TSE JADE,1.65m,1.59m,NUS,20.0,2,Women High Jump Open,A270Z07,2004-01-14 00:00:00.000,None,...,1.66945,1.6435,1.59,1.65,1.59,-0.1054,-0.07945,-0.0535,-0.14,-3.092486
14885,CHEW YIU TSE JADE,None,10.42m,NUS,20.0,4,Women Triple Jump Open,A270Z07,2004-01-14 00:00:00.000,None,...,12.98890,12.7870,10.42,10.42,10.42,-2.7708,-2.56890,-2.3670,-3.04,-17.585438
15219,CHEW YIU TSE JADE,None,5.18m,HCAA,20.0,1,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.18,5.18,5.18,-0.7196,-0.62930,-0.5390,-0.84,-8.953488
18488,CHEW YIU TSE JADE,1.55m,1.50m,Hwa Chong Alumni Association,20.0,2,Women High Jump Intermediate,None,2004-01-14,None,...,1.66945,1.6435,1.5,1.55,1.50,-0.1954,-0.16945,-0.1435,-0.23,-8.294798


In [1335]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 


In [1336]:
df_select

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,0.0,2,1500m,None,2006,SGP,...,275.65155,279.6465,295.0,295.0,295.00,-23.3434,-19.34845,-15.3535,-28.67,-5.764841
1,ONG ASHLEE YUXI,None,3.1,Singapore,0.0,2,Pole vault,None,2006,SGP,...,3.86000,3.8000,3.1,3.1,3.10,-0.8200,-0.76000,-0.7000,-0.90,-17.500000
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,0.0,5,Pole vault,None,2006,SGP,...,3.86000,3.8000,2.3,2.3,2.30,-1.6200,-1.56000,-1.5000,-1.70,-37.500000
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,0.0,4,2000m steeplechase,None,2006,SGP,...,NaN,NaN,505.2,505.2,505.20,NaN,NaN,NaN,NaN,NaN
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,0.0,5,2000m steeplechase,None,2006,SGP,...,NaN,NaN,512.2,512.2,512.20,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23.0,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,NaN,NaN,434.96,454.0,434.96,NaN,NaN,NaN,NaN,NaN
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27.0,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,NaN,NaN,458.04,458.0,458.04,NaN,NaN,NaN,NaN,NaN
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22.0,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,NaN,NaN,462.07,471.0,462.07,NaN,NaN,NaN,NaN,NaN
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25.0,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,NaN,NaN,475.82,476.0,475.82,NaN,NaN,NaN,NaN,NaN


In [1355]:
df[df['NAME']=='CHEW YIU TSE JADE']

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
13691,CHEW YIU TSE JADE,,5.26m,Hwa Chong Alumni Association,20.0,3,Women Long Jump Open,None,2004-01-14,None,...,5.80930,5.7190,5.26,5.26,5.26,-0.6396,-0.54930,-0.4590,-0.76,-7.624585
13818,CHEW YIU TSE JADE,None,5.02m,Singapore,20.0,1,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.02,5.02,5.02,-0.8796,-0.78930,-0.6990,-1.00,-11.611296
14183,CHEW YIU TSE JADE,1.55m,1.60m,SINGAPORE,20.0,2,Women High Jump Open,None,2004-01-14 00:00:00.000,None,...,1.66945,1.6435,1.6,1.55,1.60,-0.0954,-0.06945,-0.0435,-0.13,-2.514451
14192,CHEW YIU TSE JADE,5.26m,5.29m,SINGAPORE,20.0,3,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.29,5.26,5.29,-0.6096,-0.51930,-0.4290,-0.73,-7.126246
14427,CHEW YIU TSE JADE,,1.55m,National University Singapore,0.0,1,Women High Jump Open,None,None,None,...,1.66945,1.6435,1.55,1.55,1.55,-0.1454,-0.11945,-0.0935,-0.18,-5.404624
14834,CHEW YIU TSE JADE,1.65m,1.59m,NUS,20.0,2,Women High Jump Open,A270Z07,2004-01-14 00:00:00.000,None,...,1.66945,1.6435,1.59,1.65,1.59,-0.1054,-0.07945,-0.0535,-0.14,-3.092486
14885,CHEW YIU TSE JADE,None,10.42m,NUS,20.0,4,Women Triple Jump Open,A270Z07,2004-01-14 00:00:00.000,None,...,12.98890,12.7870,10.42,10.42,10.42,-2.7708,-2.56890,-2.3670,-3.04,-17.585438
15219,CHEW YIU TSE JADE,None,5.18m,HCAA,20.0,1,Women Long Jump Open,None,2004-01-14 00:00:00.000,None,...,5.80930,5.7190,5.18,5.18,5.18,-0.7196,-0.62930,-0.5390,-0.84,-8.953488
18488,CHEW YIU TSE JADE,1.55m,1.50m,Hwa Chong Alumni Association,20.0,2,Women High Jump Intermediate,None,2004-01-14,None,...,1.66945,1.6435,1.5,1.55,1.50,-0.1954,-0.16945,-0.1435,-0.23,-8.294798


In [1338]:
df_select[(df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m')]

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
6268,"Tan, Kian Ye",11.20,29.03,HENRY PARK PRIMARY SCHOOL,11.0,1,Boys 11 Year Olds 200 Meter Dash U12,nan,2013-01-12 00:00:00.000,None,...,21.7557,22.071,29.03,11.2,29.03,-7.5896,-7.2743,-6.959,-8.01,-33.106565


In [1339]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [1340]:
df_select

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,0.0,2,1500m,None,2006,SGP,...,275.65155,279.6465,295.0,295.0,295.00,-23.3434,-19.34845,-15.3535,-28.67,-5.764841
1,ONG ASHLEE YUXI,None,3.1,Singapore,0.0,2,Pole vault,None,2006,SGP,...,3.86000,3.8000,3.1,3.1,3.10,-0.8200,-0.76000,-0.7000,-0.90,-17.500000
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,0.0,5,Pole vault,None,2006,SGP,...,3.86000,3.8000,2.3,2.3,2.30,-1.6200,-1.56000,-1.5000,-1.70,-37.500000
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,0.0,4,2000m steeplechase,None,2006,SGP,...,NaN,NaN,505.2,505.2,505.20,NaN,NaN,NaN,NaN,NaN
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,0.0,5,2000m steeplechase,None,2006,SGP,...,NaN,NaN,512.2,512.2,512.20,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23.0,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,NaN,NaN,434.96,454.0,434.96,NaN,NaN,NaN,NaN,NaN
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27.0,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,NaN,NaN,458.04,458.0,458.04,NaN,NaN,NaN,NaN,NaN
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22.0,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,NaN,NaN,462.07,471.0,462.07,NaN,NaN,NaN,NaN,NaN
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25.0,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,NaN,NaN,475.82,476.0,475.82,NaN,NaN,NaN,NaN,NaN


In [1341]:
foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')


In [1342]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [1343]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1344]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [1345]:
no_foreigners_list

,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,COUNTRY,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,HSIN-WEN CLARA CHUA,None,04:55.0,Singapore,0.0,2,1500m,None,2006,SGP,...,275.65155,279.6465,295.0,295.0,295.00,-23.3434,-19.34845,-15.3535,-28.67,-5.764841
1,ONG ASHLEE YUXI,None,3.1,Singapore,0.0,2,Pole vault,None,2006,SGP,...,3.86000,3.8000,3.1,3.1,3.10,-0.8200,-0.76000,-0.7000,-0.90,-17.500000
2,LEONG JIA EN.LORRAINE,None,2.3,Singapore,0.0,5,Pole vault,None,2006,SGP,...,3.86000,3.8000,2.3,2.3,2.30,-1.6200,-1.56000,-1.5000,-1.70,-37.500000
3,TAN CADENCE ZE QING,None,08:25.2,Singapore,0.0,4,2000m steeplechase,None,2006,SGP,...,NaN,NaN,505.2,505.2,505.20,NaN,NaN,NaN,NaN,NaN
4,ANISAH BINTE OMAR ALKHATIB,None,08:32.2,Singapore,0.0,5,2000m steeplechase,None,2006,SGP,...,NaN,NaN,512.2,512.2,512.20,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18530,"Aravinth, Adarsh",7:34.00,7:14.96,NUS,23.0,1,Men 2400 Meter Run Tertiary Men,None,2001-10-13 00:00:00.000,None,...,NaN,NaN,434.96,454.0,434.96,NaN,NaN,NaN,NaN,NaN
18531,"Wen Qiang, Koh",7:38.00,7:38.04,SIT,27.0,4,Men 2400 Meter Run Tertiary Men,None,1997-04-23 00:00:00.000,None,...,NaN,NaN,458.04,458.0,458.04,NaN,NaN,NaN,NaN,NaN
18532,"Loo EnPing, Nathan",7:51.00,7:42.07,NUS,22.0,5,Men 2400 Meter Run Tertiary Men,None,2002-05-11 00:00:00.000,None,...,NaN,NaN,462.07,471.0,462.07,NaN,NaN,NaN,NaN,NaN
18533,"Jun Yang, Luah",7:56.00,7:55.82,NUS,25.0,7,Men 2400 Meter Run Tertiary Men,None,1999-05-12 00:00:00.000,None,...,NaN,NaN,475.82,476.0,475.82,NaN,NaN,NaN,NaN,NaN


In [1346]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [1347]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [1348]:
top_performers_clean.reset_index(inplace=True)


In [1349]:
top_performers_clean.to_csv('top_performers_clean.csv', encoding='utf-8')

In [1350]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,14824,"yeo, husni",None,10.09m,Singapore Polytechnic,19.0,17,Men Triple Jump Open,H132F05,2005-02-01 00:00:00.000,...,15.1505,14.915,10.09,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,13987,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18.0,9,Women Triple Jump Open,None,2006-10-30 00:00:00.000,...,12.9889,12.787,9.22,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,10594,"chua, keira",,10.16m,Cedar Girls Secondary School,16.0,5,Women Triple Jump U18,None,2008-09-18,...,12.9889,12.787,10.16,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,104,"Zhong, Chuhan",None,12.26,Singapore,0.0,nan,Triple Jump,None,18-Oct-05,...,12.9889,12.787,12.26,12.26,12.26,-0.9308,-0.7289,-0.527,-1.20,-3.915305
4,820,Yvette Lee,None,10.13,VJC,0.0,7.0,Triple Jump,None,None,...,12.9889,12.787,10.13,10.13,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,5026,"., Shaik Isa",None,3.61m,Club ZOOM,8.0,13,Boys 7-8 Shot Put 1kg Medicine U9,nan,2016-11-21 00:00:00.000,...,NaN,NaN,3.61,3.61,3.61,NaN,NaN,NaN,NaN,NaN
13726,4910,"., Nur Elena",None,3.79m,UNA,6.0,1,Girls 4-6 Shot Put 1kg Medicine U7,nan,2018-12-04 00:00:00.000,...,NaN,NaN,3.79,3.79,3.79,NaN,NaN,NaN,NaN,NaN
13727,6221,"., Josiah",None,2.94m,Seng Kang Primary School,11.0,35,Boys 11 Year Olds Long Jump U12,nan,2013-06-06 00:00:00.000,...,NaN,NaN,2.94,2.94,2.94,NaN,NaN,NaN,NaN,NaN
13728,18155,"., Augustino",None,1:07.72,SMTFA,45.0,1,M45 400 Long Hurdles (0.914m) Masters,nan,1978-08-05 00:00:00.000,...,NaN,NaN,67.72,67.72,67.72,NaN,NaN,NaN,NaN,NaN


In [1351]:
# Join 2024 best results for each event for each athlete

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/Additions/')


df_yoy = pd.read_csv("2023_performance.csv")

df_yoy['NAME'] = df_yoy['NAME'].str.replace('\xa0', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\r', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\n', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.strip()


In [1352]:
df_yoy

,EVENT,NAME,GENDER,TEAM,BEST_RESULT(2023)
0,100m,"CHUA, JOSHUA",Male,SINGAPORE,10.59
1,100m,"RAPHAEL, RYAN",Male,SINGAPORE,10.72
2,100m,"HO ANN HENG, XANDER",Male,NANYANG TECHNOLOGICAL UNI,10.73
3,100m,TATE TAN FUNG,Male,RI,10.74
4,100m,"TAN, DARYL",Male,SINGAPORE INSTITUTE OF TECHNOL,10.64
5,100m,"ONG, YING TAT",Male,VICTORIA JUNIOR COLLEGE,10.72
6,100m,"RYAN, PRAHARSH",Male,NANYANG TECHNOLOGICAL UNI,10.74
7,10000m,"LIM, JON",Male,NATIONAL UNIVERSITY SINGAPORE,32:54.40
8,110m hurdles,"CHONG, WEI GUAN",Male,NATIONAL UNIVERSITY OF SINGAPO,14.29
9,1500m,"SOUNDARARAJAH, JEEVANEESH",Male,INDIVIDUAL,4:06.52


In [1353]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

yoy_performance = pd.merge(
    left=df_yoy, 
    right=top_performers_clean,
    how='left',
    left_on=['EVENT', 'GENDER', 'NAME'],
    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],
)

In [1354]:
yoy_performance

,EVENT,NAME,GENDER,TEAM_x,BEST_RESULT(2023),index,SEED,RESULT,TEAM_y,AGE,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,100m,"CHUA, JOSHUA",Male,SINGAPORE,10.59,15933.0,None,10.83,Wings Athletics Club,24.0,...,10.808505,10.96515,10.83,10.83,10.83,-0.17814,-0.021495,0.13515,-0.387,1.294168
1,100m,"RAPHAEL, RYAN",Male,SINGAPORE,10.72,18519.0,,10.72,Wings Athletics Club,21.0,...,10.808505,10.96515,10.72,10.72,10.72,-0.06814,0.088505,0.24515,-0.277,2.347506
2,100m,"HO ANN HENG, XANDER",Male,NANYANG TECHNOLOGICAL UNI,10.73,33.0,None,10.91,Singapore,0.0,...,10.808505,10.96515,10.91,10.91,10.91,-0.25814,-0.101495,0.05515,-0.467,0.528105
3,100m,TATE TAN FUNG,Male,RI,10.74,14891.0,None,10.70,NUS,19.0,...,10.808505,10.96515,10.7,10.7,10.70,-0.04814,0.108505,0.26515,-0.257,2.539021
4,100m,"TAN, DARYL",Male,SINGAPORE INSTITUTE OF TECHNOL,10.64,13558.0,10.77,10.64,Singapore Institute of Technol,23.0,...,10.808505,10.96515,10.64,10.77,10.64,0.01186,0.168505,0.32515,-0.197,3.113569
5,100m,"ONG, YING TAT",Male,VICTORIA JUNIOR COLLEGE,10.72,322.0,None,10.68,Singapore,0.0,...,10.808505,10.96515,10.68,10.68,10.68,-0.02814,0.128505,0.28515,-0.237,2.730537
6,100m,"RYAN, PRAHARSH",Male,NANYANG TECHNOLOGICAL UNI,10.74,13559.0,10.78,10.74,Nanyang Technological Uni,26.0,...,10.808505,10.96515,10.74,10.78,10.74,-0.08814,0.068505,0.22515,-0.297,2.155990
7,10000m,"LIM, JON",Male,NATIONAL UNIVERSITY SINGAPORE,32:54.40,14591.0,,32:54.40,National University Singapore,0.0,...,1951.544250,1979.82750,1974.4,1974.4,1974.40,-51.13900,-22.855750,5.42750,-88.850,0.287847
8,110m hurdles,"CHONG, WEI GUAN",Male,NATIONAL UNIVERSITY OF SINGAPO,14.29,15188.0,None,15.62,NUS,24.0,...,14.339925,14.54775,15.62,15.62,15.62,-1.48790,-1.280075,-1.07225,-1.765,-7.739083
9,1500m,"SOUNDARARAJAH, JEEVANEESH",Male,INDIVIDUAL,4:06.52,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [410]:
yoy_performance.to_csv('yoy_performance.csv', index=False, encoding='utf-8')

In [161]:
tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [162]:
tiered_performers

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,3.5%,5%,RESULT_CONV,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
3672,13778,"Seet, Rachel",35:00.00,31:47.33,Singapore University of Social,21.0,2,Mixed 5000 Meter Race Walk Open,None,2003-08-26,...,NaN,NaN,1907.33,2100.0,1907.33,NaN,NaN,NaN,NaN,NaN
3675,13779,"Loo, Xuan Yu",,28:32.55,Oldham Athletics,17.0,1,Mixed 5000 Meter Race Walk Open,None,2007-05-01,...,NaN,NaN,1712.55,1712.55,1712.55,NaN,NaN,NaN,NaN,NaN
3679,13777,"Lim Yu Xian, Exodus",,39:45.12,Singapore University of Social,29.0,4,Mixed 5000 Meter Race Walk Open,None,1995-12-03,...,NaN,NaN,2385.12,2385.12,2385.12,NaN,NaN,NaN,NaN,NaN
3687,13780,"Cher, Shi Kai Laura",,31:50.44,Cedar Girls' Secondary School,16.0,3,Mixed 5000 Meter Race Walk Open,None,2008-09-23,...,NaN,NaN,1910.44,1910.44,1910.44,NaN,NaN,NaN,NaN,NaN
5409,17562,"{650.0: 'BINTE MOHAMMAD, ROSNAH', 60.0: 'b Abd...",,5:11.47,Erovra Club,0.0,5,Mixed 40-44 4x400 Meter Relay Masters,,None,...,NaN,NaN,311.47,311.47,311.47,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13745,17518,"ANNA, CORNEY",,2.37m,Singapore Masters Track & Fiel,57.0,1,Women 55-59 Long Jump Masters,None,1967-12-12 00:00:00.000,...,NaN,NaN,2.37,2.37,2.37,NaN,NaN,NaN,NaN,NaN
13746,9674,"ANG, ZI EN, STACIE",,4.02m,Raffles Girls' School,13.0,12,Women Long Jump U15,None,2011-04-24,...,NaN,NaN,4.02,4.02,4.02,NaN,NaN,NaN,NaN,NaN
13748,10987,"ANG, LE XUAN LEAZANNE",,24.08m,Dunman High School,7.0,3,Women Javelin Throw (500g) U18,None,2017-09-02,...,NaN,NaN,24.08,24.08,24.08,NaN,NaN,NaN,NaN,NaN
13752,11214,"ABDUL AZIZ, ZARA ALMIRA",5.03m,4.79m,Singapore Sports School,17.0,4,Girls Long Jump U18,None,2007-07-06 00:00:00.000,...,NaN,NaN,4.79,5.03,4.79,NaN,NaN,NaN,NaN,NaN


In [163]:
# Filter top 8 performances for each event and gender

#top_performers = top_performers_clean.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'],ascending=[False, False, False]).groupby(['GENDER', 'MAPPED_EVENT']).head(8).reset_index(drop=True)




In [164]:
# Convert time format for marathon and 5000m into mm:ss.00
import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(top_performers_clean)):
        
    rowIndex = top_performers_clean.index[i]

    event=top_performers_clean.iloc[rowIndex,17]
        
    
    time_base2=top_performers_clean.iloc[rowIndex,21]
    time_base3=top_performers_clean.iloc[rowIndex,22]
    time_base5=top_performers_clean.iloc[rowIndex,23]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10000m' or event=='5000m' or event=='3000m steeplechase' or event=='1500m':
        
        print(time_base2, time_base3, time_base5)

        
        
        try:
            
        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
            output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
            output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
            print(output2, output3, output5)

                    
            top_performers_clean.loc[rowIndex, '2%_timing'] = output2
            top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
            top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            

            
        except:
            
            pass
        

        
    elif event=='Marathon':
        
        print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
            top_performers_clean.loc[rowIndex, '2%_timing'] = output2
            top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
            top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             


9535.98 9676.215 9816.45
output 02:38:55 02:41:16 02:43:36
10431.54 10584.945 10738.35
output 02:53:51 02:56:24 02:58:58
10431.54 10584.945 10738.35
output 02:53:51 02:56:24 02:58:58
9535.98 9676.215 9816.45
output 02:38:55 02:41:16 02:43:36
10431.54 10584.945 10738.35
output 02:53:51 02:56:24 02:58:58
9535.98 9676.215 9816.45
output 02:38:55 02:41:16 02:43:36
9535.98 9676.215 9816.45
output 02:38:55 02:41:16 02:43:36
9535.98 9676.215 9816.45
output 02:38:55 02:41:16 02:43:36
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
116.1

116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.1372

116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
131.733 133.67024999999998 135.60750000000002
02:11.733000 02:13.670250 02:15.607500
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
116.1372 117.84509999999999 119.55300000000001
01:56.137200 01:57.845100 01:59.553000
131.733 133.67024999999998 135.60750000000002
02:11.733000 0

545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
674.067 683.97975 693.8925
11:14.067000 11:23.979750 11:33.892500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
674.067 683.97975 693.8925
11:14.067000 11:23.979750 11:33.892500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
674.067 683.97975 693.8925
11:14.067000 11:23.979750 11:33.892500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
545.751 553.7767499999999 561.8025
09:05.751000 09:13.776750 09:21.802500
674.067 683.97975 693.8925
11:14.067000 11:23.979750 11:33.892

271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 

271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.779 251.37
04:04.188000 04:07.779000 04:11.370000
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
271.65659999999997 275.65155 279.6465
04:31.656600 04:35.651550 04:39.646500
244.18800000000002 247.

1923.261 1951.5442499999997 1979.8275
32:03.261000 32:31.544250 32:59.827500
nan nan nan
nan nan nan


In [165]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,SEED_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,2%_timing,3.5%_timing,5%_timing
0,14810,"yeo, husni",None,10.09m,Singapore Polytechnic,19.0,17,Men Triple Jump Open,H132F05,2005-02-01 00:00:00.000,...,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484,NaN,NaN,NaN
1,13973,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18.0,9,Women Triple Jump Open,None,2006-10-30 00:00:00.000,...,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743,NaN,NaN,NaN
2,10580,"chua, keira",,10.16m,Cedar Girls Secondary School,16.0,5,Women Triple Jump U18,None,2008-09-18,...,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088,NaN,NaN,NaN
3,90,"Zhong, Chuhan",None,12.26,Singapore,0.0,nan,Triple Jump,None,18-Oct-05,...,12.26,12.26,-0.9308,-0.7289,-0.527,-1.20,-3.915305,NaN,NaN,NaN
4,806,Yvette Lee,None,10.13,VJC,0.0,7.0,Triple Jump,None,None,...,10.13,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13754,5012,"., Shaik Isa",None,3.61m,Club ZOOM,8.0,13,Boys 7-8 Shot Put 1kg Medicine U9,nan,2016-11-21 00:00:00.000,...,3.61,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13755,4896,"., Nur Elena",None,3.79m,UNA,6.0,1,Girls 4-6 Shot Put 1kg Medicine U7,nan,2018-12-04 00:00:00.000,...,3.79,3.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13756,6207,"., Josiah",None,2.94m,Seng Kang Primary School,11.0,35,Boys 11 Year Olds Long Jump U12,nan,2013-06-06 00:00:00.000,...,2.94,2.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13757,18141,"., Augustino",None,1:07.72,SMTFA,45.0,1,M45 400 Long Hurdles (0.914m) Masters,nan,1978-08-05 00:00:00.000,...,67.72,67.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
# Identify Tier 1/2/3 performers

top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


In [167]:
top_performers_clean

,index,NAME,SEED,RESULT,TEAM,AGE,COMPETITION_RANK,EVENT_x,UNIQUE_ID,DOB,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,2%_timing,3.5%_timing,5%_timing,TIER
0,14810,"yeo, husni",None,10.09m,Singapore Polytechnic,19.0,17,Men Triple Jump Open,H132F05,2005-02-01 00:00:00.000,...,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484,NaN,NaN,NaN,
1,13973,"ooi, chen si",,9.22m,Ngee Ann Polytechnic,18.0,9,Women Triple Jump Open,None,2006-10-30 00:00:00.000,...,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743,NaN,NaN,NaN,
2,10580,"chua, keira",,10.16m,Cedar Girls Secondary School,16.0,5,Women Triple Jump U18,None,2008-09-18,...,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088,NaN,NaN,NaN,
3,90,"Zhong, Chuhan",None,12.26,Singapore,0.0,nan,Triple Jump,None,18-Oct-05,...,12.26,-0.9308,-0.7289,-0.527,-1.20,-3.915305,NaN,NaN,NaN,
4,806,Yvette Lee,None,10.13,VJC,0.0,7.0,Triple Jump,None,None,...,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13754,5012,"., Shaik Isa",None,3.61m,Club ZOOM,8.0,13,Boys 7-8 Shot Put 1kg Medicine U9,nan,2016-11-21 00:00:00.000,...,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
13755,4896,"., Nur Elena",None,3.79m,UNA,6.0,1,Girls 4-6 Shot Put 1kg Medicine U7,nan,2018-12-04 00:00:00.000,...,3.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
13756,6207,"., Josiah",None,2.94m,Seng Kang Primary School,11.0,35,Boys 11 Year Olds Long Jump U12,nan,2013-06-06 00:00:00.000,...,2.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
13757,18141,"., Augustino",None,1:07.72,SMTFA,45.0,1,M45 400 Long Hurdles (0.914m) Masters,nan,1978-08-05 00:00:00.000,...,67.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [174]:
# Drop rows without a SEAG benchmark

final_df = top_performers_clean[top_performers_clean['BENCHMARK'].notna()]


In [175]:
final_df.to_csv('top_performers_tiered.csv', encoding='utf-8')

In [169]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one


#top_performers_final = top_performers_sorted.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, False], na_position='last')


In [3934]:
# Apply Rule 3 & is within 5% performance band

octc_df = df_top.loc[(((df_top['CATEGORY_EVENT']=='Mid')|(df_top['CATEGORY_EVENT']=='Sprint')|(df_top['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df_top['CATEGORY_EVENT']=='Steeple')|(df_top['CATEGORY_EVENT']=='Pentathlon')|(df_top['CATEGORY_EVENT']=='Heptathlon')|(df_top['CATEGORY_EVENT']=='Triathlon'))&(df_top['RESULT_CONV'] <= df_top['5%']) & (df_top['AGE']<40) & ((df_top['MAPPED_EVENT']!='Marathon')|(df_top['AGE']<60) & (df_top['MAPPED_EVENT']=='Marathon')))|(((df_top['CATEGORY_EVENT']=='Jump')|(df_top['CATEGORY_EVENT']=='Throw'))&(df_top['RESULT_CONV'] >= df_top['5%']) & (df_top['AGE']<40) & ((df_top['MAPPED_EVENT']!='Marathon')|(df_top['AGE']<60) & (df_top['MAPPED_EVENT']=='Marathon')))]

In [3935]:
octc_df

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,...,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,Delta2,Delta3.5,Delta5,PERF_SCALAR
0,"Ting Li King, Brandon",15.03m,21.0,1,Men Triple Jump Open,2002-01-09 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484
1,"ANURA, ANDRE",14.93m,24.0,2,Men Triple Jump Open,1999-06-12 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,14.93,-0.456,-0.22050,0.0150,0.095541
16,"CHEN, Yi Chu",16.87m,21.0,1,Men Shot Put Open,2002-10-31 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451
17,"RIGAN, JONAH CHANG",16.50m,20.0,2,Men Shot Put Open,2003-03-25 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.50,-0.454,-0.19450,0.0650,0.375723
20,"Loh, Anson",15.71m,15.0,1,Men Shot Put U18,2008-04-28,None,Throw,Male,52nd SA Inter Club Championships 2023,...,Shot put,17.30,16.954,16.69450,16.4350,15.71,-1.244,-0.98450,-0.7250,-4.190751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,Women 100 Meter Dash Open,1996-09-20 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.43,0.555,0.73125,0.9075,7.723404
537,"HU, Chia-Chen",11.57,26.0,2,Women 100 Meter Dash Open,1997-03-28 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915
540,"TAN, ELIZABETH-ANN",11.91,20.0,3,Women 100 Meter Dash Open,2003-09-23 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.91,0.075,0.25125,0.4275,3.638298
555,"Gurung, Subas",31:44.38,26.0,1,Men 10000 Meter Run Open,1997-10-22 00:00:00.000,None,Long,Male,SA Allcomers Meet 2,...,10000m,1885.55,1923.261,1951.54425,1979.8275,1904.38,18.881,47.16425,75.4475,4.001352


In [3936]:
octc_df

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,COMPETITION,...,EVENT_y,Metric,2%,3.5%,5%,RESULT_CONV,Delta2,Delta3.5,Delta5,PERF_SCALAR
0,"Ting Li King, Brandon",15.03m,21.0,1,Men Triple Jump Open,2002-01-09 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,15.03,-0.356,-0.12050,0.1150,0.732484
1,"ANURA, ANDRE",14.93m,24.0,2,Men Triple Jump Open,1999-06-12 00:00:00.000,None,Jump,Male,83rd Singapore Open Track & Field,...,Triple jump,15.70,15.386,15.15050,14.9150,14.93,-0.456,-0.22050,0.0150,0.095541
16,"CHEN, Yi Chu",16.87m,21.0,1,Men Shot Put Open,2002-10-31 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.87,-0.084,0.17550,0.4350,2.514451
17,"RIGAN, JONAH CHANG",16.50m,20.0,2,Men Shot Put Open,2003-03-25 00:00:00.000,None,Throw,Male,83rd Singapore Open Track & Field,...,Shot put,17.30,16.954,16.69450,16.4350,16.50,-0.454,-0.19450,0.0650,0.375723
20,"Loh, Anson",15.71m,15.0,1,Men Shot Put U18,2008-04-28,None,Throw,Male,52nd SA Inter Club Championships 2023,...,Shot put,17.30,16.954,16.69450,16.4350,15.71,-1.244,-0.98450,-0.7250,-4.190751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,"PEREIRA, VERONICA SHANTI",11.43,27.0,1,Women 100 Meter Dash Open,1996-09-20 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.43,0.555,0.73125,0.9075,7.723404
537,"HU, Chia-Chen",11.57,26.0,2,Women 100 Meter Dash Open,1997-03-28 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.57,0.415,0.59125,0.7675,6.531915
540,"TAN, ELIZABETH-ANN",11.91,20.0,3,Women 100 Meter Dash Open,2003-09-23 00:00:00.000,None,Sprint,Female,83rd Singapore Open Track & Field,...,100m,11.75,11.985,12.16125,12.3375,11.91,0.075,0.25125,0.4275,3.638298
555,"Gurung, Subas",31:44.38,26.0,1,Men 10000 Meter Run Open,1997-10-22 00:00:00.000,None,Long,Male,SA Allcomers Meet 2,...,10000m,1885.55,1923.261,1951.54425,1979.8275,1904.38,18.881,47.16425,75.4475,4.001352


In [1022]:
octc_df.to_csv('octc_df.csv', sep=',', encoding='utf-8-sig', index=False)


In [364]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


# Download list of foreigners and spex carded athletes

In [365]:
#spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [431]:
foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')


In [367]:
#spex_list

,V1,V2,V3,V4,V5,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin","QUEK, CALVIN JUN JIE",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,NaN,E3P


In [432]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN
1,Akahodani,Takayuki,NaN,NaN,NaN
2,Apondar,Audric,NaN,NaN,NaN
3,Brooks,Ruby,NaN,NaN,NaN
4,Brouwer,Cees,NaN,NaN,NaN
...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN
220,NGUYEN,HOAI VAN,NaN,NaN,NaN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN


In [433]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

In [434]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4,V1,V2,V3,V4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN,Aaryan Greuter Christoph,Greuter Christoph Aaryan,"Aaryan, Greuter Christoph",Greuter Christoph Aaryan
1,Akahodani,Takayuki,NaN,NaN,NaN,Akahodani Takayuki,Takayuki Akahodani,"Akahodani, Takayuki",Takayuki Akahodani
2,Apondar,Audric,NaN,NaN,NaN,Apondar Audric,Audric Apondar,"Apondar, Audric",Audric Apondar
3,Brooks,Ruby,NaN,NaN,NaN,Brooks Ruby,Ruby Brooks,"Brooks , Ruby",Ruby Brooks
4,Brouwer,Cees,NaN,NaN,NaN,Brouwer Cees,Cees Brouwer,"Brouwer, Cees",Cees Brouwer
...,...,...,...,...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN,CHO CHIA-HSUAN,CHIA-HSUAN CHO,"CHO, CHIA-HSUAN",CHIA-HSUAN CHO
220,NGUYEN,HOAI VAN,NaN,NaN,NaN,NGUYEN HOAI VAN,HOAI VAN NGUYEN,"NGUYEN , HOAI VAN",HOAI VAN NGUYEN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN,NGUYEN QUOC THINH,QUOC THINH NGUYEN,"NGUYEN , QUOC THINH",QUOC THINH NGUYEN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN,PHAM QUYNH GIANG,QUYNH GIANG PHAM,"PHAM , QUYNH GIANG",QUYNH GIANG PHAM


In [371]:
#spex1 = spex_list['V1'].dropna().tolist()
#spex2 = spex_list['V2'].dropna().tolist()
#spex3 = spex_list['V3'].dropna().tolist()
#spex4 = spex_list['V4'].dropna().tolist()
#spex5 = spex_list['V5'].dropna().tolist()


In [435]:
for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()


In [373]:
#spex_athletes=spex1+spex2+spex3+spex4+spex5

In [436]:
foreign_list = for1+for2+for3+for4 

In [437]:
#spex_athletes

In [438]:
foreign_list

['Aaryan Greuter Christoph',
 'Akahodani Takayuki',
 'Apondar Audric',
 'Brooks  Ruby',
 'Brouwer Cees',
 'CARLESATER ARMAAN',
 'Chary Mathangi ',
 'Cinthikael Angel',
 'Comia Tsang Hannah',
 'Comia Tsang Mhandy',
 'Donnelly Jordan ',
 'Dorai Trinity Sharilyn',
 'Freeman Alexander',
 'Gaume Constance',
 'Gregorio John Alexander',
 'HAGIWARA RYOTA ',
 'Indarto Gerard',
 'Joy Matthew',
 'JULURI ADITYA',
 'Kiet Tran',
 'Koduru Kalyani',
 'Kuchenbuch Natalya',
 'KUIJPERS KUIJPERS AYKO ELLIN',
 'Kumar Joseph Suraj',
 'KUPPUSAMY PRAKSHEETA',
 'Kusumo Naia',
 'Kuwalekar  Spruha ',
 'Lahey Kezia Stephanie ',
 'Liz Jerry',
 'Marican Raees Marican Bin Ayoob Marican',
 'MARIE BRUNO DE FERRIERES DE SAUVEBOEUF LOUIS ',
 'Maslov Timofei',
 'Masrezwan Maeva',
 'MENDOZA NICCOLO',
 'Mennella Leonardo',
 'Meyers Nikita Mae Jiny-Yu',
 'Mishra Sunay',
 'Mo Thant Cin',
 'Moon Daniel Youngjo',
 'OCAMPO ANIAG DWAYNE MICHAEL OCAMPO ANIAG',
 'Ortega Rafael Pedro',
 'Owen Elia',
 'owl emma',
 'Ozbudak Deniz',
 

In [439]:
#spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [440]:
foreign_list_casefold=[s.casefold() for s in foreign_list]

In [442]:
#spex_athletes_casefold

In [443]:
foreign_list_casefold

['aaryan greuter christoph',
 'akahodani takayuki',
 'apondar audric',
 'brooks  ruby',
 'brouwer cees',
 'carlesater armaan',
 'chary mathangi ',
 'cinthikael angel',
 'comia tsang hannah',
 'comia tsang mhandy',
 'donnelly jordan ',
 'dorai trinity sharilyn',
 'freeman alexander',
 'gaume constance',
 'gregorio john alexander',
 'hagiwara ryota ',
 'indarto gerard',
 'joy matthew',
 'juluri aditya',
 'kiet tran',
 'koduru kalyani',
 'kuchenbuch natalya',
 'kuijpers kuijpers ayko ellin',
 'kumar joseph suraj',
 'kuppusamy praksheeta',
 'kusumo naia',
 'kuwalekar  spruha ',
 'lahey kezia stephanie ',
 'liz jerry',
 'marican raees marican bin ayoob marican',
 'marie bruno de ferrieres de sauveboeuf louis ',
 'maslov timofei',
 'masrezwan maeva',
 'mendoza niccolo',
 'mennella leonardo',
 'meyers nikita mae jiny-yu',
 'mishra sunay',
 'mo thant cin',
 'moon daniel youngjo',
 'ocampo aniag dwayne michael ocampo aniag',
 'ortega rafael pedro',
 'owen elia',
 'owl emma',
 'ozbudak deniz',
 

In [444]:
# Exclusion list is foreigners + spex carded 

#exclusions = foreign_list_casefold + spex_athletes_casefold

exclusions = foreign_list_casefold

In [445]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one

top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [446]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
8353,11670,{},12.99,8.0,2,Boys 4x100 Meter Relay 15-16,2015-01-01 00:00:00.000,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.99,27.1572,27.74760,28.3380,71.996951
12480,17949,"{9: 'Ng, Caitlin Shan Wen', 438: 'Seow, Kyra',...",22.17,0.0,1,Girls 13-14 4x100 Meter Relay U15,None,None,Relay,Female,...,4 x 100m relay,44.58,45.4716,46.14030,46.8090,22.17,23.3016,23.97030,24.6390,55.269179
9267,13141,"{995: 'Lee, Asher', 991: 'Yeo, Chee Hean Phili...",29.89,0.0,2,Boys 9-12 4x400 Meter Relay U13,None,None,Relay,Male,...,4 x 400m relay,188.82,192.5964,195.42870,198.2610,29.89,162.7064,165.53870,168.3710,89.170109
9588,13486,"{994: 'De Ming, Gilbert Lee', 988: 'Lee, Jerem...",38.89,0.0,14,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,38.89,1.2572,1.84760,2.4380,6.194106
12424,17876,"{991: 'Yeo, Chee Hean Philip', 992: 'Yeo, Chee...",12.92,0.0,16,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.92,27.2272,27.81760,28.4080,72.174797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,15886,"., Khairulnazim",14.72,17.0,4,Men 400 Meter Dash Open,2006-01-27,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,14.72,32.8426,33.54205,34.2415,73.432340
9918,13873,"., Kaarthika",12.53,27.0,9,Women 400 Meter Dash Women Junior,1996-06-13,None,Sprint,Female,...,400m,53.84,54.9168,55.72440,56.5320,12.53,42.3868,43.19440,44.0020,81.727340
10701,15018,"., Jayashree",1:14.05,15.0,36,Girls Long Jump B Div,2008-01-29 00:00:00.000,None,Jump,Female,...,Long jump,6.02,5.8996,5.80930,5.7190,10.97,5.0704,5.16070,5.2510,87.225914
12077,17484,"., Cheung Zheng",47.61,14.0,20,Boys 11-14 400 Meter Dash U15,2009-02-19,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,47.61,-0.0474,0.65205,1.3515,2.898349


In [447]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)

In [448]:
excluded_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [449]:
#spexed_list=top_performers

In [450]:
excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
excluded_list['overall_rank'] = 1
excluded_list['overall_rank'] = excluded_list.groupby(['MAPPED_EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [451]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [452]:
#Apply OCTC selection rule: max 6 for 100m/400m and max 3 for all other events

excluded_list = excluded_list[(((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<7))|(~((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<4))]


In [453]:
excluded_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
8135,11374,"Tan, Bernice",2:16.97,23.0,3,Women 10000 Meter Run Open,2000-11-16,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,1
8137,11380,"NicAmhlaoibh, Maire",2:16.97,35.0,2,Women 10000 Meter Run Open,1988-02-14,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,2
8134,11372,"Ng, Yew Cheo",2:16.97,37.0,4,Women 10000 Meter Run Open,1986-11-30,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,3
9739,13637,"PHAGAMI, KHUM BAHADUR",20.34m,39.0,2,Men 10000 Meter Run Men Senior A,1984-08-09,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,1
9734,13632,"Mohd, Jamian",35.31m,38.0,6,Men 10000 Meter Run Men Senior A,1985-04-29,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,3544,SOLANA LEANN REINKIRSTEN CANDA,00:22.7,0.0,9.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,2
3260,3555,SHERYL TOH PEIXUAN,00:23.4,0.0,6.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,3
3257,3552,RAO TIANYU,00:28.7,0.0,12.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,1
3246,3541,NG JUN JIE,00:15.7,0.0,11.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,2


In [454]:
excluded_list.to_csv('octc_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?

# Marinda's request for 6 and 8 place finish analysis

In [4062]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND (RANK='3' OR RANK='6' OR RANK='8')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4064]:
SEAG.tail(60)

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
53,Farell Glen Felix Jerus,2.15 m,3,High jump,Jump,Male,SEA Games,Final
54,Sun Soklim,NM,6,High jump,Jump,Female,SEA Games,Final
55,Kobsit Sittichai,2.07 m,6,High jump,Jump,Male,SEA Games,Final
56,Suwandi Wijaya,7.47,6,Long jump,Jump,Male,SEA Games,Final
57,Pok Pisey,4.76,8,Long jump,Jump,Female,SEA Games,Final
58,Sapwaturrahman Sapwaturrahman,7.62,3,Long jump,Jump,Male,SEA Games,Final
59,Bùi Thị Loan,6.02,3,Long jump,Jump,Female,SEA Games,Final
60,Nurul Ashikin Abas,5.67,6,Long jump,Jump,Female,SEA Games,Final
61,John Marvin Aragon Rafols,7.19,8,Long jump,Jump,Male,SEA Games,Final
62,Sunisa Khotseemueang,None,3,Heptathlon,Heptathlon,Female,SEA Games,Final


In [247]:
for i in range(len(SEAG)):
        
    rowIndex = SEAG.index[i]

    input_string=SEAG.iloc[rowIndex,3]  # event is column index 3
    
    metric=SEAG.iloc[rowIndex,1] # result is column index 1
    
    if (metric==None or metric=='NH' or metric=='NM'):  # skip those non-numeric values
        continue
        
   # elif metric=='NH':
    #    continue
        
    print(i, input_string, metric)
        
    out = convert_time(i, input_string, metric)
     
    SEAG.loc[rowIndex, 'Metric'] = out

0 1500m     03:59.40
1 1500m     04:04.3
2 1500m     04:05.4
3 Decathlon 6891
4 100m 10.78
5 100m 11.75
6 100m 11.96
7 100m 10.443
8 200m 24.09
9 200m 23.6
11 200m 21.02
12 200m 21.58
13 400m 46.63
14 400m 48.26
15 400m 57.54
16 400m 1:01.20
17 400m 53.84
18 800m 1:57.98
19 800m 1:53.86
20 800m 2:15.77
21 800m 2:18.25
22 800m 2:09.15
23 800m 1:55.39
24 1500m 4:26.33
25 1500m 4:38.64
26 1500m 5:00.090
27 5000m 17:13.63
28 5000m 18:41.76
29 5000m 14:43.45
30 5000m 15:01.79
31 5000m 18:02.52
32 5000m 15:09.65
33 10000m 33:00.23
34 10000m 32:36.32
35 10000m 38:11.04
36 10000m 37:29.58
37 10000m 35:31.03
38 10000m 31:25.55
39 Marathon 2:35:49
40 Marathon 3:17:36
41 Marathon 3:46:44
42 Marathon 2:52:41
43 Marathon 2:50:27
44 Marathon 2:41:36
45 Shot put 11.69
46 Shot put 12.12
47 Shot put 14.44
48 Shot put 14.65
49 Shot put 17.3
50 Shot put 16.06
52 High jump 2.07 m
53 High jump 2.15 m
54 High jump 1.73
55 High jump 1.95 m
56 Long jump 7.47
57 Long jump 5.67
58 Long jump 6.02
59 Long jump 4.

In [248]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78
...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06


In [249]:
SEAG.to_csv('check_variation.csv', encoding='utf-8')

In [250]:
comps = SEAG[SEAG['RANK']=='3']

In [251]:
comps

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.000
5,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final,11.750
7,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final,10.443
9,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final,23.600
11,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final,21.020
13,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final,46.630
17,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final,53.840
19,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final,113.860
22,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final,129.150


In [252]:
new_comps = comps[['EVENT', 'GENDER', 'Metric']]

In [253]:
#comps=SEAG.groupby(['EVENT', 'GENDER'])['Metric'].min()


In [254]:
df2 = SEAG.merge(new_comps, on=['EVENT', 'GENDER'], how='left')


In [255]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443
...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050


In [256]:
df2.to_csv('check_metric.csv', encoding='utf-8')

In [257]:
df2['% VARIATION']=(df2['Metric_x']/df2['Metric_y']*100-100)

In [258]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y,% VARIATION
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400,0.000000
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400,2.046784
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400,2.506266
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000,0.000000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443,3.227042
...,...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290,0.000000
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850,0.000000
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050,0.000000
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050,18.878609


In [259]:
df2.to_csv('SEAG_variation.csv', sep=',', encoding='utf-8-sig', index=False)


# Convert timing format from seconds to HMSf

In [4803]:
datetime.datetime.fromtimestamp(120).strftime('%M:%S.%f')
        

'32:00.000000'

In [4808]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [5295]:
import datetime
def format_result(result):
    date = datetime.datetime.utcfromtimestamp(result)
    output = datetime.datetime.strftime(date, "%H:%M:%S.%f")
    return output
print(format_result(113.86))

00:01:53.860000


# Clean manual load data

In [178]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/Additions/')


df_SHA = pd.read_csv("SHA_Additions_2024.csv", encoding='latin-1')

In [179]:
df_SHA

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,NaN,NaN,"TAN, AARON JUSTIN",Individual,NaN,2:35:26,NaN,NaN,NaN,NaN,...,1993.0,NaN,Marathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,"MELVIN, WONG",Individual,NaN,2:46:31,NaN,NaN,NaN,NaN,...,1983.0,NaN,Marathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,SOH RUI YONG GUILLAUME,Individual,NaN,31:11.4,NaN,NaN,NaN,NaN,...,1991.0,NaN,Long,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,"OSMAN, AMIR RUSYAIDI",Individual,NaN,04:11.9,NaN,NaN,NaN,NaN,...,2002.0,NaN,Mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"TAN, YONG JIN, KERWIN",Individual,NaN,2:49:39,NaN,NaN,NaN,NaN,...,NaN,NaN,Marathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,DEVATHAS SATIANATHANÊ,Individual,NaN,2:56:03,NaN,NaN,NaN,NaN,...,NaN,NaN,Marathon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,LIM YEE CHERN CLARA,Singapore,NaN,12.95,q,NaN,NaN,-1.3,...,2006.0,NaN,Sprint,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,NaN,HSIN-WEN CLARA CHUA,Singapore,NaN,04:55.0,NaN,NaN,NaN,NaN,...,2006.0,NaN,Mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.0,NaN,TEH YING SHAN,Singapore,NaN,12.71,Q,NaN,NaN,-0.7,...,2006.0,NaN,Sprint,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2.0,NaN,ONG ASHLEE YUXI,Singapore,NaN,3.1,NaN,NaN,NaN,NaN,...,2006.0,NaN,Jump,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [181]:
df_SHA.to_csv('SHA_data.csv', index=False, encoding='utf-8')